📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
  - [Image Degradation](#toc2_1_)    
- [Image Enhancement](#toc3_)    
  - [Frequency Domain](#toc3_1_)    
    - [Discrete Fourier Transform (DFT)](#toc3_1_1_)    
      - [1-Dimensional Signal](#toc3_1_1_1_)    
        - [Basis Vectors](#toc3_1_1_1_1_)    
        - [Forward Discrete Fourier Transform (DFT)](#toc3_1_1_1_2_)    
        - [Inverse Discrete Fourier Transform (IDFT)](#toc3_1_1_1_3_)    
      - [2-Dimensional Signal](#toc3_1_1_2_)    
        - [Basis Images](#toc3_1_1_2_1_)    
        - [Forward 2D Discrete Fourier Transform (DFT2)](#toc3_1_1_2_2_)    
        - [Inverse 2D Discrete Fourier Transform (IDFT2)](#toc3_1_1_2_3_)    
      - [Basic Examples](#toc3_1_1_3_)    
        - [Dirac Delta (Impulse)](#toc3_1_1_3_1_)    
        - [Constant (DC) Signal](#toc3_1_1_3_2_)    
        - [Rectangular Pulse](#toc3_1_1_3_3_)    
        - [Sine and Cosine Waves](#toc3_1_1_3_4_)    
        - [Gaussian](#toc3_1_1_3_5_)    
        - [Vertical Line](#toc3_1_1_3_6_)    
        - [Horizontal Line](#toc3_1_1_3_7_)    
        - [Diagonal Line](#toc3_1_1_3_8_)    
      - [Fourier Transform vs. Circular Convolution](#toc3_1_1_4_)    
        - [Properly Padding Signal & Filter](#toc3_1_1_4_1_)    
        - [Periodicity](#toc3_1_1_4_2_)    
        - [Converting Spatial-Domain Filters to Frequency-Domain](#toc3_1_1_4_3_)    
      - [Filtering](#toc3_1_1_5_)    
        - [Low-Pass Filters](#toc3_1_1_5_1_)    
        - [High-Pass Filters](#toc3_1_1_5_2_)    
        - [Band-Pass Filters](#toc3_1_1_5_3_)    
        - [Band-Reject Filters](#toc3_1_1_5_4_)    
      - [Effects on Fourier Spectrum](#toc3_1_1_6_)    
        - [Flip](#toc3_1_1_6_1_)    
        - [Circular Shift](#toc3_1_1_6_2_)    
        - [Rotate](#toc3_1_1_6_3_)    
      - [Examples](#toc3_1_1_7_)    
        - [Example 1: Exploring Ringing Effect](#toc3_1_1_7_1_)    
        - [Example 2: Image Smoothing](#toc3_1_1_7_2_)    
        - [Example 3: Image Sharpenning](#toc3_1_1_7_3_)    
        - [Example 4: Edge Detection](#toc3_1_1_7_4_)    
        - [Example 5: Periodic Noise Removal](#toc3_1_1_7_5_)    
        - [Example 6: Incomplete Reconstruction](#toc3_1_1_7_6_)    
    - [Discrete Cosine Transform (DCT)](#toc3_1_2_)    
      - [2-Dimensional Signal](#toc3_1_2_1_)    
        - [Even-Symmetric Signal](#toc3_1_2_1_1_)    
        - [Basis Images](#toc3_1_2_1_2_)    
        - [Forward 2D Discrete Cosine Transform (DCT2)](#toc3_1_2_1_3_)    
        - [Inverse 2D Discrete Cosine Transform (IDCT2)](#toc3_1_2_1_4_)    
      - [Blocking Effect](#toc3_1_2_2_)    
    - [Energy of the Signals](#toc3_1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from matplotlib.gridspec import GridSpec
from PIL import Image

In [ ]:
rng = np.random.default_rng(seed=42)

In [ ]:
np.set_printoptions(linewidth=160)

# <a id='toc2_'></a>[Load Images](#toc0_)

In [ ]:
im_1 = cv2.imread(
    "../assets/images/dip_3rd/CH02_Fig0222(b)(cameraman).tif",
    cv2.IMREAD_GRAYSCALE,
)

im_2 = cv2.cvtColor(
    cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif"),
    cv2.COLOR_BGR2RGB,
)


# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), layout="compressed")
axs[0].imshow(im_1, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("CH02_Fig0222(b)(cameraman).tif")
axs[1].imshow(im_2, vmin=0, vmax=255)
axs[1].set_title("CH06_Fig0638(a)(lenna_RGB).tif")
plt.show()

## <a id='toc2_1_'></a>[Image Degradation](#toc0_)

- Check out the [**image-degredation.ipynb**](./utils/image-degredation.ipynb) notebook for more information on the topic of degredation.


In [ ]:
def apply_averaging_filter(img: np.ndarray, kernel_size: int) -> np.ndarray:
    kernel = np.ones((kernel_size, kernel_size)) / kernel_size**2
    return cv2.filter2D(img, -1, kernel, borderType=cv2.BORDER_CONSTANT)

In [ ]:
def apply_gaussian_noise(img: np.ndarray, mean: float = 0, std: float = 25) -> np.ndarray:
    gaussian_noise = rng.normal(loc=mean, scale=std, size=img.shape)
    return np.clip(img.astype(np.float64) + gaussian_noise, 0, 255).astype(np.uint8)

In [ ]:
def apply_periodic_noise(image: np.ndarray, u0: int, v0: int, noise_intensity: float = 0.5) -> np.ndarray:
    image_float = image / np.iinfo(image.dtype).max
    height, width = image.shape[:2]

    x = np.linspace(0, 1, width, endpoint=False)
    y = np.linspace(0, 1, height, endpoint=False)
    X, Y = np.meshgrid(x, y)

    periodic_noise = np.sin(2 * np.pi * (u0 * X + v0 * Y))
    periodic_noise = periodic_noise / np.max(np.abs(periodic_noise))

    if len(image.shape) == 2:
        noisy_image_float = image_float + noise_intensity * periodic_noise
    elif len(image.shape) == 3:
        noisy_image_float = image_float + noise_intensity * periodic_noise[:, :, np.newaxis]

    noisy_image_float = np.clip(noisy_image_float, 0, 1)
    noisy_image = (noisy_image_float * 255).astype(np.uint8)

    return noisy_image

In [ ]:
im_1_blurred = apply_averaging_filter(im_1, kernel_size=3)
im_2_blurred = apply_averaging_filter(im_2, kernel_size=3)
im_1_gaussian_noise = apply_gaussian_noise(im_1, mean=0, std=20)
im_2_gaussian_noise = apply_gaussian_noise(im_2, mean=0, std=50)
im_1_periodic_noise = apply_periodic_noise(im_1, u0=20, v0=10)
im_2_periodic_noise = apply_periodic_noise(im_2, u0=40, v0=20)

# plot
images = [
    [im_1, im_1_blurred, im_1_gaussian_noise, im_1_periodic_noise],
    [im_2, im_2_blurred, im_2_gaussian_noise, im_2_periodic_noise],
]
titles = [
    ["Original", "Blurred", "Gaussian Noise", "Periodic Noise"],
    ["Original", "Blurred", "Gaussian Noise", "Periodic Noise"],
]
cmaps = [["gray"] * 4, [None] * 4]
fig, axs = plt.subplots(2, 4, figsize=(16, 8), layout="compressed")
for i in range(2):
    for j in range(4):
        axs[i, j].imshow(images[i][j], cmap=cmaps[i][j], vmin=0, vmax=255)
        axs[i, j].set_title(titles[i][j])
        axs[i, j].axis("off")
plt.show()

# <a id='toc3_'></a>[Image Enhancement](#toc0_)

Image enhancement is the procedure of improving the quality for a specific purpose!

- Spatial Domain
  - Intensity Transformation
  - Histogram Processing
  - Spatial Filtering
- Frequency Domain
  - **Fourier Transform**
  - **Cosine Transform**
- Spatial-Frequency Domain
  - Wavelet Transform (Multi-resolution Analysis)


## <a id='toc3_1_'></a>[Frequency Domain](#toc0_)

<table style="margin:0 auto;">
  <thead>
    <tr>
      <th>Aspect</th>
      <th>Fast Discrete Fourier Transform (FFT)</th>
      <th>Discrete Cosine Transform (DCT)</th>
      <th>Notes</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Domain</strong></td>
      <td>Complex frequency domain</td>
      <td>Real frequency domain</td>
      <td>FFT handles both magnitude and phase; DCT simplifies processing</td>
    </tr>
    <tr>
      <td><strong>Computational Complexity</strong></td>
      <td>O(N log N)</td>
      <td>O(N log N)</td>
      <td>Both are efficient; choice depends on signal characteristics</td>
    </tr>
    <tr>
      <td><strong>Energy Compaction</strong></td>
      <td>Less effective</td>
      <td>High compaction</td>
      <td>DCT is favored in compression (e.g., JPEG)</td>
    </tr>
    <tr>
      <td><strong>Symmetry &amp; Boundary Conditions</strong></td>
      <td>Assumes periodic extension</td>
      <td>Assumes even symmetry</td>
      <td>DCT minimizes boundary artifacts in images</td>
    </tr>
    <tr>
      <td><strong>Coefficient Type</strong></td>
      <td>Complex (real &amp; imaginary)</td>
      <td>Real</td>
      <td>Simpler storage and processing for image data</td>
    </tr>
    <tr>
      <td><strong>Applications in DIP</strong></td>
      <td>
        <ul style="margin: 0; padding-left: 20px;">
          <li>Spectral analysis</li>
          <li>Frequency filtering</li>
          <li>Convolution-based methods</li>
          <li>Phase correlation (image registration)</li>
        </ul>
      </td>
      <td>
        <ul style="margin: 0; padding-left: 20px;">
          <li>Image compression (e.g., JPEG)</li>
          <li>Feature extraction</li>
          <li>Texture &amp; pattern analysis</li>
          <li>Artifact reduction</li>
        </ul>
      </td>
      <td>Use FFT for detailed frequency features and DCT for compression and reduced artifacts</td>
    </tr>
  </tbody>
</table>

📝 **Docs**:

- `numpy.fft`: [numpy.org/doc/stable/reference/routines.fft.html](http://numpy.org/doc/stable/reference/routines.fft.html)
- `scipy.fft`: [docs.scipy.org/doc/scipy/reference/fft.html](https://docs.scipy.org/doc/scipy/reference/fft.html)
- Colormap reference [`matplotlib`]: [matplotlib.org/stable/gallery/color/colormap_reference.html](https://matplotlib.org/stable/gallery/color/colormap_reference.html)
- Check out the [**transform.ipynb**](./utils/transform.ipynb) notebook for comprehensive information on the topic of frequency transforms.

### <a id='toc3_1_1_'></a>[Discrete Fourier Transform (DFT)](#toc0_)


#### <a id='toc3_1_1_1_'></a>[1-Dimensional Signal](#toc0_)


##### <a id='toc3_1_1_1_1_'></a>[Basis Vectors](#toc0_)

Instead of computing each frequency separately, we can represent DFT as matrix multiplication.

**Mathematical Formula:**

- the DFT matrix $W$:
$$W_{k, x} = e^{-i 2 \pi \frac{kx}{N}}$$

- For $N=5$, the DFT matrix is:
$$W_k =
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 \\
1 & e^{-i \frac{2\pi}{5}} & e^{-i \frac{4\pi}{5}} & e^{-i \frac{6\pi}{5}} & e^{-i \frac{8\pi}{5}} \\
1 & e^{-i \frac{4\pi}{5}} & e^{-i \frac{8\pi}{5}} & e^{-i \frac{12\pi}{5}} & e^{-i \frac{16\pi}{5}} \\
1 & e^{-i \frac{6\pi}{5}} & e^{-i \frac{12\pi}{5}} & e^{-i \frac{18\pi}{5}} & e^{-i \frac{24\pi}{5}} \\
1 & e^{-i \frac{8\pi}{5}} & e^{-i \frac{16\pi}{5}} & e^{-i \frac{24\pi}{5}} & e^{-i \frac{32\pi}{5}}
\end{bmatrix}
$$


In [ ]:
def dft_basis_vectors(length: int, mode: str = "efficient") -> np.ndarray:
    if mode == "inefficient":
        W = np.zeros((length, length), dtype=complex)
        for k in range(length):
            for x in range(length):
                W[k, x] = np.exp(-2j * np.pi * k * x / length)
    elif mode == "efficient":
        k = np.arange(length)
        x = k[:, None]
        W = np.exp(-2j * np.pi * k * x / length)

    return W

In [ ]:
signal_length = 5
basis_vectors = dft_basis_vectors(signal_length)

# log
print(f"basis vectors:\n{basis_vectors}")

In [ ]:
# time values for sinusoidal representation
t = np.linspace(0, signal_length - 1, 100)

# plot
fig, axs = plt.subplots(3, signal_length, figsize=(18, 8), layout="compressed")
for i in range(signal_length):
    axs[0, i].stem(np.real(basis_vectors)[i], linefmt="y-", markerfmt="yo", basefmt="r-")
    axs[0, i].set_title(f"Basis {i}")
    axs[0, i].set_ylabel("Amplitude [Real]")
    axs[0, i].set_xlabel("Index")
    axs[0, i].grid(True, linewidth=0.5, linestyle="--", color="gray")
    axs[1, i].stem(np.imag(basis_vectors)[i], linefmt="c-", markerfmt="co", basefmt="r-")
    axs[1, i].set_ylabel("Amplitude [Imag]")
    axs[1, i].set_xlabel("Index")
    axs[1, i].grid(True, linewidth=0.5, linestyle="--", color="gray")
    sinusoidal_wave = np.exp(-2j * np.pi * i * t / signal_length)  # continuous version
    axs[2, i].plot(t, np.real(sinusoidal_wave), "y-", label="Real")
    axs[2, i].plot(t, np.imag(sinusoidal_wave), "c--", label="Imaginary")
    axs[2, i].legend()
    axs[2, i].set_ylabel("Amplitude")
    axs[2, i].set_xlabel("Time")
    axs[2, i].grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

##### <a id='toc3_1_1_1_2_'></a>[Forward Discrete Fourier Transform (DFT)](#toc0_)

$$F = W \cdot f$$

📈 **Magnitude**:

- The magnitude of each frequency component indicates how **strong** or **dominant** that frequency is in the original signal.
- A larger magnitude means the frequency component is **more significant**.
- A magnitude of $0$ means that the corresponding frequency component is **absent** from the signal — in which case its phase is mathematically defined but **not meaningful**.

$$|F(k)| = \sqrt{\text{Re}(F(k))^2 + \text{Im}(F(k))^2}$$

🔁 **Phase**: 

- The phase of each frequency component tells us how much the sinusoidal wave at that specific frequency is **circularly shifted** in time.
- A phase of $0$ means the sinusoidal wave is **aligned** with the start of the signal (i.e., no shift).
- A nonzero phase indicates a **delay** or **advance**, depending on its sign, which adjusts the basis wave to best match the signal.
- A phase of $\pi$ (or $-\pi$) means the sinusoidal wave is **flipped** (i.e., shifted by half a cycle).
- **Note**: Phase is only meaningful when the corresponding magnitude is **nonzero**; otherwise, it has no interpretive value.

$$\theta(k) = \arg(F(k)) = \tan^{-1} \left( \frac{\text{Im}(F(k))}{\text{Re}(F(k))} \right)$$

**DC and AC Components:**

- **DC (Direct Current):**
  - The **DC component** of the signal corresponds to the **zero-frequency** component of the DFT.
  - It represents the **average (mean) value** of the signal over time.
  - The DC component is the magnitude of the DFT at frequency \( f = 0 \).

- **AC (Alternating Current):**
  - The **AC components** correspond to the **non-zero frequency** components of the DFT.
  - These represent the **oscillatory** or **time-varying** parts of the signal.
  - Each AC component captures a specific periodic variation (frequency) present in the signal.


In [ ]:
def dft(f: np.ndarray) -> np.ndarray:
    W = dft_basis_vectors(length=len(f))
    F = W @ f
    return F

In [ ]:
input_1d = np.array([1, 2, 3, 2, 1])
input_1d_dft = dft(input_1d)

input_1d_dft_real = np.real(input_1d_dft)
input_1d_dft_imag = np.imag(input_1d_dft)
input_1d_dft_abs = np.abs(input_1d_dft)
input_1d_dft_phase = np.angle(input_1d_dft)

# log
print(f"input_1d_dft   : {input_1d_dft}")
print(f"real part      : {input_1d_dft_real}")
print(f"imaginary part : {input_1d_dft_imag}")
print(f"magnitude      : {input_1d_dft_abs}")
print(f"phase          : {input_1d_dft_phase}")

In [ ]:
# plot
fig, axs = plt.subplots(1, 3, figsize=(16, 4), layout="compressed")
axs[0].stem(input_1d)
axs[0].set(title="Input Signal", xlabel="Sample Index", ylabel="Amplitude", xticks=range(5))
axs[0].grid(True, linewidth=0.5, linestyle="--", color="gray")
axs[1].stem(input_1d_dft_abs)
axs[1].set(title="Magnitude of Spectrum", xlabel="Frequency Index", ylabel="Magnitude", xticks=range(5))
axs[1].grid(True, linewidth=0.5, linestyle="--", color="gray")
axs[2].stem(input_1d_dft_phase)
axs[2].set(title="Phase of Spectrum", xlabel="Frequency Index", ylabel="Phase (radians)", xticks=range(5))
axs[2].grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

##### <a id='toc3_1_1_1_3_'></a>[Inverse Discrete Fourier Transform (IDFT)](#toc0_)

$$f = W^* \cdot F \cdot \frac{1}{N}$$


In [ ]:
def idft(F: np.ndarray) -> np.ndarray:
    W_CT = np.conj(dft_basis_vectors(length=len(F))).T
    f = (W_CT @ F) / len(F)
    return f

In [ ]:
input_1d_idft = idft(input_1d_dft)
input_1d_idft_real = np.real(input_1d_idft)

# log
print(f"input_1d_idft      : {input_1d_idft}")
print(f"input_1d_idft_real : {input_1d_idft_real}")
print(f"Exact Values:")
for i, v in enumerate(input_1d_idft_real):
    print(f"\tinput_1d_idft_real[{i}]: {v.item()}")

#### <a id='toc3_1_1_2_'></a>[2-Dimensional Signal](#toc0_)

##### <a id='toc3_1_1_2_1_'></a>[Basis Images](#toc0_)

we now have two sets of basis vectors—one corresponding to the rows and the other to the columns.

**Mathematical Formula:**

- Let $f(x,y)$ be a 2D signal of size $N \times M$. The 2D DFT matrix $W$ can be computed as:
$$W_{k_1,k_2,x,y}​=e^{−i2 \pi(\frac{k_1x}{N}+\frac{k_2y}{M}​)}$$

- For $N=4, M=5$, the DFT matrix is:
$$W_{k_1, k_2} =
\begin{bmatrix}
W_{0, 0} & W_{0, 1} & \cdots & W_{0, M-1} \\
W_{1, 0} & W_{1, 1} & \cdots & W_{1, M-1} \\
\vdots & \vdots & \ddots & \vdots \\
W_{N-1, 0} & W_{N-1, 1} & \cdots & W_{N-1, M-1}
\end{bmatrix}, \quad
W_{0,0} =
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

- We can build the **2D basis images** by taking the **outer product** of the **1D basis vectors** for **rows** and **columns**.
  - 1D DFT Basis Vector (Row Direction):
    $$v_1(k_1, x) = e^{-i 2 \pi \frac{k_1 x}{N}},  \quad x = 0, 1, \dots, N-1$$

  - 1D DFT Basis Vector (Column Direction):
    $$v_2(k_2, y) = e^{-i 2 \pi \frac{k_2 y}{M}},  \quad y = 0, 1, \dots, M-1$$

  - Constructing the 2D Basis Images:
    $$W_{k_1, k_2}(x, y) = v_1(k_1, x) \cdot v_2(k_2, y) = e^{-i 2\pi \left( \frac{k_1 x}{N} + \frac{k_2 y}{M} \right)}$$

**Example**

Consider the following $3 \times 4$ signal:

$$
S = 
\begin{bmatrix}
s_{0, 0} & s_{0, 1} & s_{0, 2} & s_{0, 3} \\
s_{1, 0} & s_{1, 1} & s_{1, 2} & s_{1, 3} \\
s_{2, 0} & s_{2, 1} & s_{2, 2} & s_{2, 3}
\end{bmatrix}
$$

The goal is to find the basis image $W_{1, 2}$, corresponding to the frequency pair $k_1 = 1$ and $k_2 = 2$.

- **Step 1**: 1D Basis Vectors
  - Row Basis Vector $v_1(k_1, x)$ for $k_1 = 1$:
    - The row basis vector for $k_1 = 1$ is given by:
    $$
    v_1(1, x) = e^{-i 2 \pi \frac{1 \cdot x}{3}} \quad \text{for } x = 0, 1, 2
    $$
    This results in:
    $$
    v_1(1, x) = \begin{bmatrix}
    e^{-i 2 \pi \frac{1 \cdot 0}{3}} \\
    e^{-i 2 \pi \frac{1 \cdot 1}{3}} \\
    e^{-i 2 \pi \frac{1 \cdot 2}{3}}
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    e^{-i \frac{2\pi}{3}} \\
    e^{-i \frac{4\pi}{3}}
    \end{bmatrix}
    $$

  - Column Basis Vector $v_2(k_2, y)$ for $k_2 = 2$:
    - The column basis vector for $k_2 = 2$ is given by:
    $$
    v_2(2, y) = e^{-i 2 \pi \frac{2 \cdot y}{4}} \quad \text{for } y = 0, 1, 2, 3
    $$
    This results in:
    $$
    v_2(2, y) = \begin{bmatrix}
    e^{-i 2 \pi \frac{2 \cdot 0}{4}} \\
    e^{-i 2 \pi \frac{2 \cdot 1}{4}} \\
    e^{-i 2 \pi \frac{2 \cdot 2}{4}} \\
    e^{-i 2 \pi \frac{2 \cdot 3}{4}}
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    e^{-i \pi} \\
    e^{-i 2\pi} \\
    e^{-i 3\pi}
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    -1 \\
    1 \\
    -1
    \end{bmatrix}
    $$

- **Step 2**: Outer Product of 1D Basis Vectors
  - To compute the 2D basis image $W_{1, 2}(x, y)$, we take the outer product of $v_1(1, x)$ and $v_2(2, y)$:
    $$
    W_{1, 2}(x, y) = v_1(1, x) \cdot v_2(2, y)
    $$

    The result of the outer product is:

    $$
    W_{1, 2} = 
    \begin{bmatrix}
    1 \cdot 1 & 1 \cdot (-1) & 1 \cdot 1 & 1 \cdot (-1) \\
    e^{-i \frac{2\pi}{3}} \cdot 1 & e^{-i \frac{2\pi}{3}} \cdot (-1) & e^{-i \frac{2\pi}{3}} \cdot 1 & e^{-i \frac{2\pi}{3}} \cdot (-1) \\
    e^{-i \frac{4\pi}{3}} \cdot 1 & e^{-i \frac{4\pi}{3}} \cdot (-1) & e^{-i \frac{4\pi}{3}} \cdot 1 & e^{-i \frac{4\pi}{3}} \cdot (-1)
    \end{bmatrix}
    $$

    Simplifying each term:

    $$
    W_{1, 2} = 
    \begin{bmatrix}
    1 & -1 & 1 & -1 \\
    e^{-i \frac{2\pi}{3}} & -e^{-i \frac{2\pi}{3}} & e^{-i \frac{2\pi}{3}} & -e^{-i \frac{2\pi}{3}} \\
    e^{-i \frac{4\pi}{3}} & -e^{-i \frac{4\pi}{3}} & e^{-i \frac{4\pi}{3}} & -e^{-i \frac{4\pi}{3}}
    \end{bmatrix}
    $$


In [ ]:
def dft_basis_images(rows: int, cols: int, mode: str = "efficient") -> np.ndarray:
    row_basis = dft_basis_vectors(rows)  # 1D basis vectors for rows
    col_basis = dft_basis_vectors(cols)  # 1D basis vectors for columns

    if mode == "inefficient":
        basis_images = np.zeros((rows, cols, rows, cols), dtype=complex)  # 4D array to store the 2D basis images
        for k1 in range(rows):
            for k2 in range(cols):
                row_vector = row_basis[k1, :]
                col_vector = col_basis[k2, :]
                basis_images[k1, k2] = np.outer(row_vector, col_vector)

    elif mode == "efficient":
        basis_images = row_basis[:, None, :, None] * col_basis[None, :, None, :]

    return basis_images

In [ ]:
signal_row = 3
signal_col = 4
basis_images = dft_basis_images(signal_row, signal_col)

# log
print(f"basis_images.shape: {basis_images.shape}")
print(f"W(0,0):\n{basis_images[0, 0]}")

In [ ]:
signal_row = 8
signal_col = 8
basis_images = dft_basis_images(signal_row, signal_col)

# plot
fig, axes = plt.subplots(signal_row, 2 * signal_col, figsize=(20, 10), layout="compressed")
for i in range(signal_row):
    for j in range(signal_col):
        basis_image = basis_images[i, j]
        axes[i, j].imshow(basis_image.real, cmap="gray", vmin=-1, vmax=1)
        axes[i, j].axis("off")
        axes[i, j + signal_col].imshow(basis_image.imag, cmap="gray", vmin=-1, vmax=1)
        axes[i, j + signal_col].axis("off")
for j in range(signal_col):
    axes[0, j].set_title(f"Real {j+1}", fontsize=8)
    axes[0, j + signal_col].set_title(f"Imag {j+1}", fontsize=8)
plt.show()

In [ ]:
N = basis_images.shape[0]
M = basis_images.shape[1]
cross_correlation = np.einsum("ijkl,mnkl->ijmn", basis_images, np.conj(basis_images), optimize=True)

# plot
fig, axes = plt.subplots(N, M, figsize=(M * 2, N * 2 + 1), layout="compressed")
for k1 in range(N):
    for l1 in range(M):
        axes[k1, l1].imshow(np.abs(cross_correlation[k1, l1]), cmap="gray")
        axes[k1, l1].set(title=f"({k1},{l1})", xticks=[], yticks=[])
        axes[k1, l1].set_xticks(np.arange(-0.5, cross_correlation.shape[3], 1), minor=True)
        axes[k1, l1].set_yticks(np.arange(-0.5, cross_correlation.shape[2], 1), minor=True)
        axes[k1, l1].grid(True, which="minor", color="white", linestyle="--", linewidth=0.5, alpha=0.5)
fig.suptitle("Cross-Correlation Between 8x8 DFT Basis Images")
plt.show()

##### <a id='toc3_1_1_2_2_'></a>[Forward 2D Discrete Fourier Transform (DFT2)](#toc0_)

- The 2D DFT can be broken down into two 1D DFTs (one along the rows and one along the columns).
$$F = W_N \cdot f \cdot W_M^T$$

In [ ]:
def dft2(f: np.ndarray, mode: str = "efficient") -> np.ndarray:
    rows, cols = f.shape

    if mode == "inefficient":
        basis_images = dft_basis_images(rows, cols)
        F = np.zeros((rows, cols), dtype=complex)
        for k1 in range(rows):
            for k2 in range(cols):
                F[k1, k2] = np.sum(f * basis_images[k1, k2])

    elif mode == "efficient":
        W_row = dft_basis_vectors(rows)
        W_col = dft_basis_vectors(cols)
        F = W_row @ f @ W_col.T

    return F

##### <a id='toc3_1_1_2_3_'></a>[Inverse 2D Discrete Fourier Transform (IDFT2)](#toc0_)

$$f = \frac{1}{N \times M} \cdot W_N^* \cdot F \cdot W_M^{*T}$$


In [ ]:
def idft2(F: np.ndarray, mode: str = "efficient") -> np.ndarray:
    rows, cols = F.shape
    normalization_factor = 1 / (rows * cols)

    if mode == "inefficient":
        basis_images = dft_basis_images(rows, cols)
        f = np.zeros((rows, cols), dtype=complex)
        for k1 in range(rows):
            for k2 in range(cols):
                f[k1, k2] = normalization_factor * np.sum(F * np.conj(basis_images[k1, k2]))

    elif mode == "efficient":
        W_row = dft_basis_vectors(rows)
        W_col = dft_basis_vectors(cols)
        f = normalization_factor * (W_row.conj().T @ F @ W_col.conj())

    return f

#### <a id='toc3_1_1_3_'></a>[Basic Examples](#toc0_)


In [ ]:
signal_h, signal_w = (128, 128)

##### <a id='toc3_1_1_3_1_'></a>[Dirac Delta (Impulse)](#toc0_)

$$f(x,y) = \delta(x-x_0,y-y_0)$$


In [ ]:
sig_delta = np.zeros(shape=(signal_h, signal_w))
sig_delta[0, 0] = 1

# spatial to frequency domain
sig_delta_fft = np.fft.fft2(sig_delta)

# magnitude & phase
sig_delta_fft_mag = np.abs(sig_delta_fft)
sig_delta_fft_pha = np.angle(sig_delta_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_delta, cmap="gray")
ax0.set(title="f(x,y)")
ax0.annotate("", xy=(5, 5), xytext=(15, 15), arrowprops=dict(facecolor="red", shrink=0.05), color="red")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_delta_fft.real, cmap="gray", vmin=0, vmax=1)
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_delta_fft.imag, cmap="gray", vmin=0, vmax=1)
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_delta_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_delta_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_2_'></a>[Constant (DC) Signal](#toc0_)

$$f(x,y)=1$$

In [ ]:
sig_dc = np.ones(shape=(signal_h, signal_w))

# spatial to frequency domain
sig_dc_fft = np.fft.fftshift(np.fft.fft2(sig_dc))

# magnitude & phase
sig_dc_fft_mag = np.abs(sig_dc_fft)
sig_dc_fft_pha = np.angle(sig_dc_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_dc, cmap="gray", vmin=0, vmax=1)
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_dc_fft.real, cmap="gray", vmin=0, vmax=1)
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_dc_fft.imag, cmap="gray", vmin=0, vmax=1)
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_dc_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_dc_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_3_'></a>[Rectangular Pulse](#toc0_)

$$
f(x, y) = \begin{cases}
1 & \text{if } |x| \leq a/2 \text{ and } |y| \leq b/2, \\
0 & \text{otherwise}.
\end{cases}
$$


In [ ]:
rect_h, rect_w = (16, 16)
sig_rectangle = np.zeros(shape=(signal_h, signal_w))
sig_rectangle[:rect_h, :rect_w] = 1

# spatial to frequency domain
sig_rectangle_fft = np.fft.fftshift(np.fft.fft2(sig_rectangle))

# magnitude & phase
sig_rectangle_fft_mag = np.abs(sig_rectangle_fft)
sig_rectangle_fft_pha = np.angle(sig_rectangle_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_rectangle, cmap="gray", vmin=0, vmax=1)
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_rectangle_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_rectangle_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(sig_rectangle_fft_mag, cmap="gray")
ax3.set(title="FFT - Magnitude", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_rectangle_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_4_'></a>[Sine and Cosine Waves](#toc0_)

$$f(x, y) = \sin(2\pi (u_0 x + v_0 y)) \quad \text{or} \quad \cos(2\pi (u_0 x + v_0 y))$$

⚠️ **Notes:**

- For a **real-valued signal**, the FFT output is **conjugate symmetric**. This means:
  $$F(u,v)=F^*(-u,-v) \quad \to \quad |F(u,v)|=|F^*(-u,-v)|$$


In [ ]:
u0 = 10  # frequency in the x-direction
v0 = 5  # frequency in the y-direction

x = np.linspace(0, 1, signal_w, endpoint=False)
y = np.linspace(0, 1, signal_h, endpoint=False)
X, Y = np.meshgrid(x, y)
sig_sine = np.sin(2 * np.pi * (u0 * X + v0 * Y))

# spatial to frequency domain
sig_sine_fft = np.fft.fft2(sig_sine)

# magnitude & phase
sig_sine_fft_mag = np.abs(sig_sine_fft)
sig_sine_fft_pha = np.angle(sig_sine_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_sine, cmap="gray", vmin=0, vmax=1)
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_sine_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_sine_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_sine_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_sine_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_5_'></a>[Gaussian](#toc0_)

$$f(x, y) = \frac{1}{2 \pi \sigma_x \sigma_y} \exp\left(-\frac{1}{2} \left(\frac{x^2}{\sigma_x^2} + \frac{y^2}{\sigma_y^2}\right)\right)$$


In [ ]:
kernel_gaussian_1 = sp.signal.windows.gaussian(signal_w, std=8)
kernel_gaussian_2 = sp.signal.windows.gaussian(signal_h, std=8)
sig_gaussian = np.outer(kernel_gaussian_1, kernel_gaussian_2)
sig_gaussian /= sig_gaussian.sum()

# spatial to frequency domain
sig_gaussian_fft = np.fft.fftshift(np.fft.fft2(sig_gaussian))

# magnitude & phase
sig_gaussian_fft_mag = np.abs(sig_gaussian_fft)
sig_gaussian_fft_pha = np.angle(sig_gaussian_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_gaussian, cmap="gray")
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_gaussian_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_gaussian_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_gaussian_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_gaussian_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_6_'></a>[Vertical Line](#toc0_)


In [ ]:
sig_vertical_line = np.zeros(shape=(signal_h, signal_w))
sig_vertical_line[:, 32:64] = 1

# spatial to frequency domain
sig_vertical_line_fft = np.fft.fftshift(np.fft.fft2(sig_vertical_line))

# magnitude & phase
sig_vertical_line_fft_mag = np.abs(sig_vertical_line_fft)
sig_vertical_line_fft_pha = np.angle(sig_vertical_line_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_vertical_line, cmap="gray")
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_vertical_line_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_vertical_line_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_vertical_line_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_vertical_line_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_7_'></a>[Horizontal Line](#toc0_)


In [ ]:
sig_horizontal_line = np.zeros(shape=(signal_h, signal_w))
sig_horizontal_line[64:96, :] = 1

# spatial to frequency domain
sig_horizontal_line_fft = np.fft.fftshift(np.fft.fft2(sig_horizontal_line))

# magnitude & phase
sig_horizontal_line_fft_mag = np.abs(sig_horizontal_line_fft)
sig_horizontal_line_fft_pha = np.angle(sig_horizontal_line_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_horizontal_line, cmap="gray")
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_horizontal_line_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_horizontal_line_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_horizontal_line_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_horizontal_line_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_3_8_'></a>[Diagonal Line](#toc0_)


In [ ]:
sig_diagonal_line = np.eye(signal_h, signal_w)

# spatial to frequency domain
sig_diagonal_line_fft = np.fft.fftshift(np.fft.fft2(sig_diagonal_line))

# magnitude & phase
sig_diagonal_line_fft_mag = np.abs(sig_diagonal_line_fft)
sig_diagonal_line_fft_pha = np.angle(sig_diagonal_line_fft)

# plot
fig = plt.figure(figsize=(18, 10), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[:, 0])
ax0.imshow(sig_diagonal_line, cmap="gray")
ax0.set(title="f(x,y)")
ax1 = fig.add_subplot(gs[0, 1])
ax1.imshow(sig_diagonal_line_fft.real, cmap="gray")
ax1.set(title="FFT - Real Part", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[0, 2])
ax2.imshow(sig_diagonal_line_fft.imag, cmap="gray")
ax2.set(title="FFT - Imaginary Part", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(sig_diagonal_line_fft_mag + 1e-12), cmap="gray")
ax3.set(title="FFT - Magnitude (dB)", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[1, 2])
ax4.imshow(sig_diagonal_line_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
ax4.set(title="FFT - Phase", xticks=[], yticks=[])
plt.show()

#### <a id='toc3_1_1_4_'></a>[Fourier Transform vs. Circular Convolution](#toc0_)

- **Circular convolution** assumes signals are periodic and wraps around at the boundaries.
- Convolution in the time domain corresponds to multiplication in the frequency domain:
  
$$
x[n] \circledast h[n] \;\xleftrightarrow{\mathcal{F}}\; X[k] \cdot H[k]
$$

- Multiplication in the time domain corresponds to circular convolution in the frequency domain:

$$
x[n] \cdot h[n] \;\xleftrightarrow{\mathcal{F}}\; X[k] \circledast H[k]
$$


In [ ]:
signal_h, signal_w = im_1.shape
kernel_h, kernel_w = (11, 11)

In [ ]:
kernel_average_11x11 = np.ones((kernel_h, kernel_w)) / (kernel_h * kernel_w)

# log
print(kernel_average_11x11)

##### <a id='toc3_1_1_4_1_'></a>[Properly Padding Signal & Filter](#toc0_)


In [ ]:
fft_h = signal_h + kernel_h - 1
fft_w = signal_w + kernel_w - 1

im_1_padded = np.pad(
    im_1,
    pad_width=((0, fft_h - signal_h), (0, fft_w - signal_w)),
)

kernel_average_11x11_padded = np.pad(
    kernel_average_11x11,
    pad_width=((0, fft_h - kernel_h), (0, fft_w - kernel_h)),
)

# log
print(f"im_1_padded.shape               : {im_1_padded.shape}")
print(f"kernel_average_11x11_padded.shape : {kernel_average_11x11_padded.shape}")

# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="compressed")
axs[0].imshow(im_1_padded, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("im_1_padded")
axs[1].imshow(kernel_average_11x11_padded, cmap="gray")
axs[1].set_title("kernel_average_11x11_padded")
plt.show()

##### <a id='toc3_1_1_4_2_'></a>[Periodicity](#toc0_)


In [ ]:
signal_periodic = np.tile(im_1_padded, (4, 4))
kernel_periodic = np.tile(kernel_average_11x11_padded, (4, 4))

# plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6), layout="compressed")
axes[0].imshow(signal_periodic, cmap="gray")
axes[0].set(title="Periodic Extension of Signal", xticks=[], yticks=[])
axes[1].imshow(kernel_periodic, cmap="gray")
axes[1].set(title="Periodic Extension of Kernel", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_4_3_'></a>[Converting Spatial-Domain Filters to Frequency-Domain](#toc0_)


In [ ]:
signal_h, signal_w = im_1.shape
kernel_h, kernel_w = (3, 3)
fft_h, fft_w = signal_h + kernel_h - 1, signal_w + kernel_w - 1

# spatial kernels
kernel_average_3x3 = np.ones((kernel_h, kernel_w)) / (kernel_h * kernel_w)
kernel_gaussian_3x3 = np.outer(sp.signal.windows.gaussian(kernel_h, 0.6), sp.signal.windows.gaussian(kernel_w, 0.6))
kernel_laplace = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
kernel_sobel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

# log
print(f"kernel_average_3x3:\n{kernel_average_3x3}\n")
print(f"kernel_gaussian_3x3:\n{kernel_gaussian_3x3}\n")
print(f"kernel_laplace:\n{kernel_laplace}\n")
print(f"kernel_sobel:\n{kernel_sobel}\n")

In [ ]:
# pad kernels
kernel_average_3x3_p = np.pad(kernel_average_3x3, pad_width=((0, fft_h - kernel_h), (0, fft_w - kernel_w)))
kernel_gaussian_3x3_p = np.pad(kernel_gaussian_3x3, pad_width=((0, fft_h - kernel_h), (0, fft_w - kernel_w)))
kernel_laplace_p = np.pad(kernel_laplace, pad_width=((0, fft_h - kernel_h), (0, fft_w - kernel_w)))
kernel_sobel_p = np.pad(kernel_sobel, pad_width=((0, fft_h - kernel_h), (0, fft_w - kernel_w)))

# transform kernels into frequency domain
kernel_average_3x3_p_fft = np.fft.fftshift(np.fft.fft2(kernel_average_3x3_p))
kernel_gaussian_3x3_p_fft = np.fft.fftshift(np.fft.fft2(kernel_gaussian_3x3_p))
kernel_laplace_p_fft = np.fft.fftshift(np.fft.fft2(kernel_laplace_p))
kernel_sobel_p_fft = np.fft.fftshift(np.fft.fft2(kernel_sobel_p))

In [ ]:
# plot
kernels = [kernel_average_3x3_p_fft, kernel_gaussian_3x3_p_fft, kernel_laplace_p_fft, kernel_sobel_p_fft]
titles = ["Average Filter", "Gaussian Filter", "Laplace Filter", "Sobel Filter"]
fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(20, 16), layout="compressed")
for i in range(len(kernels)):
    axs[i, 0].imshow(kernels[i].real, cmap="gray")
    axs[i, 0].set(title=f"{titles[i]} - real", xticks=[], yticks=[])
    axs[i, 1].imshow(kernels[i].imag, cmap="gray")
    axs[i, 1].set(title=f"{titles[i]} - imag", xticks=[], yticks=[])
    axs[i, 2].imshow(np.abs(kernels[i]), cmap="gray")
    axs[i, 2].set(title=f"{titles[i]} - magnitude", xticks=[], yticks=[])
    axs[i, 3].imshow(20 * np.log10(np.abs(kernels[i]) + 1e-12), cmap="gray")
    axs[i, 3].set(title=f"{titles[i]} - magnitude (dB)", xticks=[], yticks=[])
    axs[i, 4].imshow(np.angle(kernels[i]), cmap="twilight", vmin=-np.pi, vmax=np.pi)
    axs[i, 4].set(title=f"{titles[i]} - phase", xticks=[], yticks=[])
plt.show()

#### <a id='toc3_1_1_5_'></a>[Filtering](#toc0_)

- When you define a filter **directly in the frequency domain**, you are **bypassing** the **spatial domain entirely**.
- The filter is already defined in the frequency domain, so **there is no spatial kernel** to convolve with the image.
- There is no need to apply **padding** in order **to prevent aliasing problem** in the **Circular Convolution**.

<table style="margin:0 auto;">
  <thead>
    <tr>
      <th>Filter Type</th>
      <th>Low-Pass (LPF)</th>
      <th>High-Pass (HPF)</th>
      <th>Band-Pass (BPF)</th>
      <th>Band-Reject (BRF)</th>
      <th>Notes</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><b>Ideal</b></td>
      <td>✅</td>
      <td>✅ (1 - ILPF)</td>
      <td>✅</td>
      <td>✅ (1 - IBPF)</td>
      <td>Sharp cutoff; non-realistic</td>
    </tr>
    <tr>
      <td><b>Gaussian</b></td>
      <td>✅</td>
      <td>✅ (1 - GLPF)</td>
      <td>✅</td>
      <td>✅ (1 - GBPF)</td>
      <td>Most commonly used; smooth response</td>
    </tr>
    <tr>
      <td><b>Butterworth</b></td>
      <td>✅</td>
      <td>✅ (1 - BLPF)</td>
      <td>✅</td>
      <td>✅ (1 - BBPF)</td>
      <td>Adjustable sharpness with order</td>
    </tr>
    <tr>
      <td><b>Exponential</b></td>
      <td>✅</td>
      <td>✅ (1 - ELPF)</td>
      <td>✅</td>
      <td>✅ (1 - EBPF)</td>
      <td>Gradual decay; less common</td>
    </tr>
    <tr>
      <td><b>Trapezoidal</b></td>
      <td>✅</td>
      <td>✅ (1 - TLPF)</td>
      <td>✅</td>
      <td>✅ (1 - TBPF)</td>
      <td>Customizable transition regions</td>
    </tr>
    <tr>
      <td><b>Laplacian</b></td>
      <td>❌</td>
      <td>✅</td>
      <td>❌</td>
      <td>❌</td>
      <td>Second-order derivative filter for edge detection</td>
    </tr>
    <tr>
      <td><b>Difference of Gaussians</b></td>
      <td>✅ (Two LPFs)</td>
      <td>❌</td>
      <td>✅ (DoG = LPF1 - LPF2)</td>
      <td>❌</td>
      <td>Used in edge and texture detection</td>
    </tr>
    <tr>
      <td><b>Gabor</b></td>
      <td>❌</td>
      <td>❌</td>
      <td>✅ (Localized BPF)</td>
      <td>❌</td>
      <td>Used for texture analysis & feature extraction</td>
    </tr>
    <tr>
      <td><b>Log-Gabor</b></td>
      <td>❌</td>
      <td>❌</td>
      <td>✅ (Improved BPF)</td>
      <td>❌</td>
      <td>Better frequency localization than standard Gabor</td>
    </tr>
    <tr>
      <td><b>Notch</b></td>
      <td>❌</td>
      <td>❌</td>
      <td>❌</td>
      <td>✅</td>
      <td>Rejects specific frequency bands (e.g., periodic noise removal)</td>
    </tr>
  </tbody>
</table>


##### <a id='toc3_1_1_5_1_'></a>[Low-Pass Filters](#toc0_)


In [ ]:
# ideal low-pass filter (ILPF)
def ideal_low_pass_filter(shape: tuple[int, int], cutoff: int | float) -> np.ndarray:
    """
    H(u,v) = { 1, if D(u,v) <= cutoff
             { 0, otherwise

    where D(u,v) = sqrt((u - c_c)^2 + (v - c_r)^2)
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2)
    return (D <= cutoff).astype(np.float32)


# example
filter_ilpf_1 = ideal_low_pass_filter(im_1.shape, cutoff=4)
filter_ilpf_2 = ideal_low_pass_filter(im_1.shape, cutoff=16)
filter_ilpf_3 = ideal_low_pass_filter(im_1.shape, cutoff=32)
filter_ilpf_4 = ideal_low_pass_filter(im_1.shape, cutoff=64)

# plot
kernels = [filter_ilpf_1, filter_ilpf_2, filter_ilpf_3, filter_ilpf_4]
titles = ["cutoff=4", "cutoff=16", "cutoff=32", "cutoff=64"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gaussian low-pass filter (GLPF)
def gaussian_low_pass_filter(shape: tuple[int, int], sigma: int | float) -> np.ndarray:
    """
    H(u,v) = exp(-D(u,v)^2 / (2 * sigma^2))

    where D(u,v) = (u - c_c)^2 + (v - c_r)^2
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D2 = (x - ccol) ** 2 + (y - crow) ** 2
    return np.exp(-D2 / (2 * sigma**2)).astype(np.float32)


# example
filter_glpf_1 = gaussian_low_pass_filter(im_1.shape, sigma=4)
filter_glpf_2 = gaussian_low_pass_filter(im_1.shape, sigma=8)
filter_glpf_3 = gaussian_low_pass_filter(im_1.shape, sigma=16)
filter_glpf_4 = gaussian_low_pass_filter(im_1.shape, sigma=32)

# plot
kernels = [filter_glpf_1, filter_glpf_2, filter_glpf_3, filter_glpf_4]
titles = ["sigma=4", "sigma=8", "sigma=16", "sigma=32"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# butterworth low-pass filter (BLPF)
def butterworth_low_pass_filter(shape: tuple[int, int], cutoff: int | float, order: int = 2) -> np.ndarray:
    """
    H(u,v) = 1 / (1 + (D(u,v)/cutoff)^(2 * order))

    where D(u,v) = sqrt((u - c_c)^2 + (v - c_r)^2)
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2)
    return 1 / (1 + (D / cutoff) ** (2 * order)).astype(np.float32)


# example
filter_blpf_1 = butterworth_low_pass_filter(im_1.shape, cutoff=16, order=2)
filter_blpf_2 = butterworth_low_pass_filter(im_1.shape, cutoff=64, order=2)
filter_blpf_3 = butterworth_low_pass_filter(im_1.shape, cutoff=16, order=8)
filter_blpf_4 = butterworth_low_pass_filter(im_1.shape, cutoff=64, order=8)

# plot
kernels = [filter_blpf_1, filter_blpf_2, filter_blpf_3, filter_blpf_4]
titles = ["cutoff=16, order=2", "cutoff=64, order=2", "cutoff=16, order=8", "cutoff=64, order=8"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# exponential low-pass filter (ELPF)
def exponential_low_pass_filter(shape: tuple[int, int], cutoff: int | float) -> np.ndarray:
    """
    H(u,v) = exp(-D(u,v) / cutoff)

    where D(u,v) = sqrt((u - c_c)^2 + (v - c_r)^2)
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2)
    return np.exp(-D / cutoff).astype(np.float32)


# example
filter_elpf_1 = exponential_low_pass_filter(im_1.shape, cutoff=8)
filter_elpf_2 = exponential_low_pass_filter(im_1.shape, cutoff=16)
filter_elpf_3 = exponential_low_pass_filter(im_1.shape, cutoff=32)
filter_elpf_4 = exponential_low_pass_filter(im_1.shape, cutoff=64)

# plot
kernels = [filter_elpf_1, filter_elpf_2, filter_elpf_3, filter_elpf_4]
titles = ["cutoff=8", "cutoff=16", "cutoff=32", "cutoff=64"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
def trapezoidal_low_pass_filter(shape: tuple[int, int], inner_cutoff: float, outer_cutoff: float) -> np.ndarray:
    """
    H(u,v) = 1, if D(u,v) <= inner_cutoff
             (outer_cutoff - D(u,v)) / (outer_cutoff - inner_cutoff), if inner_cutoff < D(u,v) < outer_cutoff
             0, if D(u,v) >= outer_cutoff

    where D(u,v) = sqrt((u - c_c)^2 + (v - c_r)^2)
    """
    if inner_cutoff >= outer_cutoff:
        raise ValueError("inner_cutoff must be less than outer_cutoff")

    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2)

    H = np.zeros_like(D, dtype=np.float32)
    H[D <= inner_cutoff] = 1.0
    transition = np.logical_and(D > inner_cutoff, D < outer_cutoff)
    H[transition] = (outer_cutoff - D[transition]) / (outer_cutoff - inner_cutoff)
    return H


# example
filter_tlpf_1 = trapezoidal_low_pass_filter(im_1.shape, inner_cutoff=8, outer_cutoff=32)
filter_tlpf_2 = trapezoidal_low_pass_filter(im_1.shape, inner_cutoff=8, outer_cutoff=64)
filter_tlpf_3 = trapezoidal_low_pass_filter(im_1.shape, inner_cutoff=32, outer_cutoff=64)
filter_tlpf_4 = trapezoidal_low_pass_filter(im_1.shape, inner_cutoff=32, outer_cutoff=128)


# plot
kernels = [filter_tlpf_1, filter_tlpf_2, filter_tlpf_3, filter_tlpf_4]
titles = ["icutoff=8, ocutoff=32", "icutoff=8, ocutoff=64", "icutoff=32, ocutoff=64", "icutoff=32, ocutoff=128"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

##### <a id='toc3_1_1_5_2_'></a>[High-Pass Filters](#toc0_)


In [ ]:
# ideal high-pass filter (IHPF)
def ideal_high_pass_filter(shape: tuple[int, int], cutoff: int | float) -> np.ndarray:
    """
    H(u,v) = 1.0 - ideal_low_pass_filter(u,v)
    """
    ILPF = ideal_low_pass_filter(shape, cutoff)
    IHPF = 1.0 - ILPF
    return IHPF


# example
filter_ihpf_1 = ideal_high_pass_filter(im_1.shape, cutoff=4)
filter_ihpf_2 = ideal_high_pass_filter(im_1.shape, cutoff=16)
filter_ihpf_3 = ideal_high_pass_filter(im_1.shape, cutoff=32)
filter_ihpf_4 = ideal_high_pass_filter(im_1.shape, cutoff=64)

# plot
kernels = [filter_ihpf_1, filter_ihpf_2, filter_ihpf_3, filter_ihpf_4]
titles = ["cutoff=4", "cutoff=16", "cutoff=32", "cutoff=64"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gaussian high-pass filter (GHPF)
def gaussian_high_pass_filter(shape: tuple[int, int], sigma: int | float) -> np.ndarray:
    """
    H(u,v) = 1.0 - gaussian_low_pass_filter(u,v)
    """
    GLPF = gaussian_low_pass_filter(shape, sigma)
    GHPF = 1.0 - GLPF
    return GHPF


# example
filter_ghpf_1 = gaussian_high_pass_filter(im_1.shape, sigma=4)
filter_ghpf_2 = gaussian_high_pass_filter(im_1.shape, sigma=8)
filter_ghpf_3 = gaussian_high_pass_filter(im_1.shape, sigma=16)
filter_ghpf_4 = gaussian_high_pass_filter(im_1.shape, sigma=32)

# plot
kernels = [filter_ghpf_1, filter_ghpf_2, filter_ghpf_3, filter_ghpf_4]
titles = ["sigma=4", "sigma=8", "sigma=16", "sigma=32"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# butterworth high-pass filter (BHPF)
def butterworth_high_pass_filter(shape: tuple[int, int], cutoff: int | float, order=2) -> np.ndarray:
    """
    H(u,v) = 1.0 - butterworth_low_pass_filter
    """
    BLPF = butterworth_low_pass_filter(shape, cutoff, order)
    BHPF = 1.0 - BLPF
    return BHPF


# example
filter_bhpf_1 = butterworth_high_pass_filter(im_1.shape, cutoff=16, order=2)
filter_bhpf_2 = butterworth_high_pass_filter(im_1.shape, cutoff=64, order=2)
filter_bhpf_3 = butterworth_high_pass_filter(im_1.shape, cutoff=16, order=8)
filter_bhpf_4 = butterworth_high_pass_filter(im_1.shape, cutoff=64, order=8)

# plot
kernels = [filter_bhpf_1, filter_bhpf_2, filter_bhpf_3, filter_bhpf_4]
titles = ["cutoff=16, order=2", "cutoff=64, order=2", "cutoff=16, order=8", "cutoff=64, order=8"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# exponential high-pass filter (EHPF)
def exponential_high_pass_filter(shape: tuple[int, int], cutoff: int | float) -> np.ndarray:
    """
    H(u,v) = 1.0 - exponential_low_pass_filter
    """
    ELPF = exponential_low_pass_filter(shape, cutoff)
    EHPF = 1.0 - ELPF
    return EHPF


# example
filter_ehpf_1 = exponential_high_pass_filter(im_1.shape, cutoff=8)
filter_ehpf_2 = exponential_high_pass_filter(im_1.shape, cutoff=16)
filter_ehpf_3 = exponential_high_pass_filter(im_1.shape, cutoff=32)
filter_ehpf_4 = exponential_high_pass_filter(im_1.shape, cutoff=64)

# plot
kernels = [filter_ehpf_1, filter_ehpf_2, filter_ehpf_3, filter_ehpf_4]
titles = ["cutoff=8", "cutoff=16", "cutoff=32", "cutoff=64"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# trapezoidal high-pass filter (THPF)
def trapezoidal_high_pass_filter(shape: tuple[int, int], inner_cutoff: float, outer_cutoff: float) -> np.ndarray:
    """
    H(u,v) = 1.0 - trapezoidal_low_pass_filter(u,v)
    """
    TLPF = trapezoidal_low_pass_filter(shape, inner_cutoff, outer_cutoff)
    THPF = 1.0 - TLPF
    return THPF


# example
filter_thpf_1 = trapezoidal_high_pass_filter(im_1.shape, inner_cutoff=8, outer_cutoff=32)
filter_thpf_2 = trapezoidal_high_pass_filter(im_1.shape, inner_cutoff=8, outer_cutoff=64)
filter_thpf_3 = trapezoidal_high_pass_filter(im_1.shape, inner_cutoff=32, outer_cutoff=64)
filter_thpf_4 = trapezoidal_high_pass_filter(im_1.shape, inner_cutoff=32, outer_cutoff=128)

# plot
kernels = [filter_thpf_1, filter_thpf_2, filter_thpf_3, filter_thpf_4]
titles = ["icutoff=8, ocutoff=32", "icutoff=8, ocutoff=64", "icutoff=32, ocutoff=64", "icutoff=32, ocutoff=128"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# laplacian high-pass filter
def laplacian_high_pass_filter(shape: tuple[int, int]) -> np.ndarray:
    """
    H(u,v) = -4 * pi^2 * ( (u - c_c)^2 + (v - c_r)^2 )
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D2 = (x - ccol) ** 2 + (y - crow) ** 2
    H = -4 * np.pi**2 * D2
    H /= np.max(np.abs(H))  # normalize to [-1, 0]
    return H.astype(np.float32)


# example
filter_lhpf = laplacian_high_pass_filter(im_1.shape)

# plot
plt.figure(figsize=(4, 4), layout="compressed")
plt.imshow(-filter_lhpf, cmap="gray")
plt.show()

##### <a id='toc3_1_1_5_3_'></a>[Band-Pass Filters](#toc0_)


In [ ]:
# ideal band-pass filter (IBPF)
def ideal_band_pass_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    H(u,v) = H_LPF(u,v; D2) - H_LPF(u,v; D1)
    """
    H_low_D2 = ideal_low_pass_filter(shape, D2)
    H_low_D1 = ideal_low_pass_filter(shape, D1)
    H_band = H_low_D2 - H_low_D1
    return H_band


# example
filter_ibpf_1 = ideal_band_pass_filter(im_1.shape, D1=16, D2=64)
filter_ibpf_2 = ideal_band_pass_filter(im_1.shape, D1=32, D2=64)
filter_ibpf_3 = ideal_band_pass_filter(im_1.shape, D1=32, D2=128)
filter_ibpf_4 = ideal_band_pass_filter(im_1.shape, D1=64, D2=128)

# plot
kernels = [filter_ibpf_1, filter_ibpf_2, filter_ibpf_3, filter_ibpf_4]
titles = ["D1=16, D2=64", "D1=32, D2=64", "D1=32, D2=128", "D1=64, D2=128"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gaussian band-pass filter (GBPF)
def gaussian_band_pass_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    H(u,v) = H_LPF(u,v; D2) - H_LPF(u,v; D1)
    """
    H_low_D2 = gaussian_low_pass_filter(shape, D2)
    H_low_D1 = gaussian_low_pass_filter(shape, D1)
    H_band = H_low_D2 - H_low_D1
    return H_band


# example
filter_gbpf_1 = gaussian_band_pass_filter(im_1.shape, D1=16, D2=32)
filter_gbpf_2 = gaussian_band_pass_filter(im_1.shape, D1=32, D2=64)
filter_gbpf_3 = gaussian_band_pass_filter(im_1.shape, D1=32, D2=64)
filter_gbpf_4 = gaussian_band_pass_filter(im_1.shape, D1=64, D2=100)

# plot
kernels = [filter_gbpf_1, filter_gbpf_2, filter_gbpf_3, filter_gbpf_4]
titles = ["D1=16, D2=32", "D1=32, D2=64", "D1=32, D2=64", "D1=64, D2=100"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# butterworth band-pass filter (BBPF)
def butterworth_band_pass_filter(shape: tuple[int, int], D1: float, D2: float, order: int = 2) -> np.ndarray:
    """
    H(u,v) = H_LPF(u,v; D2) - H_LPF(u,v; D1)
    """
    H_low_D2 = butterworth_low_pass_filter(shape, D2, order)
    H_low_D1 = butterworth_low_pass_filter(shape, D1, order)
    H_band = H_low_D2 - H_low_D1
    return H_band


# example
filter_bbpf_1 = butterworth_band_pass_filter(im_1.shape, D1=16, D2=32, order=2)
filter_bbpf_2 = butterworth_band_pass_filter(im_1.shape, D1=32, D2=64, order=2)
filter_bbpf_3 = butterworth_band_pass_filter(im_1.shape, D1=32, D2=64, order=8)
filter_bbpf_4 = butterworth_band_pass_filter(im_1.shape, D1=64, D2=100, order=8)

# plot
kernels = [filter_bbpf_1, filter_bbpf_2, filter_bbpf_3, filter_bbpf_4]
titles = ["D1=16, D2=32, order=2", "D1=32, D2=64, order=2", "D1=32, D2=64, order=8", "D1=64, D2=100, order=8"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# exponential band-pass filter (EBPF)
def exponential_band_pass_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    H(u,v) = H_LPF(u,v; D2) - H_LPF(u,v; D1)
    """
    H_low_D2 = exponential_low_pass_filter(shape, D2)
    H_low_D1 = exponential_low_pass_filter(shape, D1)
    H_band = H_low_D2 - H_low_D1
    return H_band


# example
filter_ebpf_1 = exponential_band_pass_filter(im_1.shape, D1=16, D2=32)
filter_ebpf_2 = exponential_band_pass_filter(im_1.shape, D1=32, D2=64)
filter_ebpf_3 = exponential_band_pass_filter(im_1.shape, D1=32, D2=64)
filter_ebpf_4 = exponential_band_pass_filter(im_1.shape, D1=64, D2=100)

# plot
kernels = [filter_ebpf_1, filter_ebpf_2, filter_ebpf_3, filter_ebpf_4]
titles = ["D1=16, D2=32", "D1=32, D2=64", "D1=32, D2=64", "D1=64, D2=100"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# trapezoidal band-pass filter (TBPF)
def trapezoidal_band_pass_filter(shape: tuple[int, int], D1: float, D2: float, D3: float, D4: float) -> np.ndarray:
    """
    H(u,v) = H_LPF(u,v; D4, D3) - H_LPF(u,v; D1, D2)
    """
    H_outer = trapezoidal_low_pass_filter(shape, D3, D4)
    H_inner = trapezoidal_low_pass_filter(shape, D1, D2)
    H_band = H_outer - H_inner
    return H_band


# example
filter_tbpf_1 = trapezoidal_band_pass_filter(im_1.shape, D1=8, D2=16, D3=32, D4=64)
filter_tbpf_2 = trapezoidal_band_pass_filter(im_1.shape, D1=16, D2=32, D3=40, D4=72)
filter_tbpf_3 = trapezoidal_band_pass_filter(im_1.shape, D1=16, D2=32, D3=64, D4=100)
filter_tbpf_4 = trapezoidal_band_pass_filter(im_1.shape, D1=32, D2=64, D3=76, D4=112)

# plot
kernels = [filter_tbpf_1, filter_tbpf_2, filter_tbpf_3, filter_tbpf_4]
titles = [
    "D1=8, D2=16, D3=32, D4=64",
    "D1=16, D2=32, D3=40, D4=72",
    "D1=16, D2=32, D3=64, D4=100",
    "D1=32, D2=64, D3=76, D4=112",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# difference of gaussians (DoG)
def difference_of_gaussians(shape: tuple[int, int], sigma1: float, sigma2: float) -> np.ndarray:
    """
    H(u,v) = G(u,v; sigma1) - G(u,v; sigma2)

    where:
    G(u,v; sigma) = exp(-D(u,v)^2 / (2 * sigma^2))
    D(u,v) = sqrt((u - c_c)^2 + (v - c_r)^2)

    Typically, sigma1 < sigma2 to create a band-pass effect.
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D2 = (x - ccol) ** 2 + (y - crow) ** 2
    H1 = np.exp(-D2 / (2 * sigma1**2))
    H2 = np.exp(-D2 / (2 * sigma2**2))
    H = H1 - H2
    return H.astype(np.float32)


# example
filter_dbpf_1 = difference_of_gaussians(im_1.shape, sigma1=8, sigma2=16)
filter_dbpf_2 = difference_of_gaussians(im_1.shape, sigma1=16, sigma2=32)
filter_dbpf_3 = difference_of_gaussians(im_1.shape, sigma1=16, sigma2=64)
filter_dbpf_4 = difference_of_gaussians(im_1.shape, sigma1=32, sigma2=64)

# plot
kernels = [filter_dbpf_1, filter_dbpf_2, filter_dbpf_3, filter_dbpf_4]
titles = ["sigma1=8, sigma2=16", "sigma1=16, sigma2=32", "sigma1=16, sigma2=64", "sigma1=32, sigma2=64"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(-kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gabor filter
def gabor_filter(shape: tuple[int, int], sigma: float, theta: float, lambda_: float, psi: float = 0) -> np.ndarray:
    """
    H(x,y) = exp(-(x_theta^2 + y_theta^2) / (2*sigma^2)) * cos(2*pi*x_theta/lambda_ + psi)

    where:
    x_theta = (x - c_c) * cos(theta) + (y - c_r) * sin(theta)
    y_theta = -(x - c_c) * sin(theta) + (y - c_r) * cos(theta)

    Parameters:
    - sigma: Gaussian envelope standard deviation
    - theta: orientation angle (radians)
    - lambda_: wavelength of the sinusoidal factor
    - psi: phase offset (default 0)
    """
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    x_theta = (x - ccol) * np.cos(theta) + (y - crow) * np.sin(theta)
    y_theta = -(x - ccol) * np.sin(theta) + (y - crow) * np.cos(theta)
    gaussian = np.exp(-(x_theta**2 + y_theta**2) / (2 * sigma**2))
    sinusoidal = np.cos(2 * np.pi * x_theta / lambda_ + psi)
    return (gaussian * sinusoidal).astype(np.float32)


# example
filter_gabor_1 = gabor_filter(im_1.shape, sigma=4, theta=0, lambda_=10, psi=0)
filter_gabor_2 = gabor_filter(im_1.shape, sigma=6, theta=np.pi / 4, lambda_=8, psi=0)
filter_gabor_3 = gabor_filter(im_1.shape, sigma=3, theta=np.pi / 2, lambda_=12, psi=np.pi / 2)
filter_gabor_4 = gabor_filter(im_1.shape, sigma=5, theta=3 * np.pi / 4, lambda_=6, psi=0)

# plot
kernels = [filter_gabor_1, filter_gabor_2, filter_gabor_3, filter_gabor_4]
titles = [
    "sigma=4, theta=0, lambda=10, psi=0",
    "sigma=6, theta=pi/4, lambda=8, psi=0",
    "sigma=3, theta=pi/2, lambda=12, psi=pi/2",
    "sigma=5, theta=3*pi/4, lambda=6, psi=0",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# log-gabor filter
def log_gabor_filter(shape: tuple[int, int], center_freq: float, bandwidth: float) -> np.ndarray:
    rows, cols = shape
    crow, ccol = rows // 2, cols // 2
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((x - ccol) ** 2 + (y - crow) ** 2)
    D = np.maximum(D, 1e-12)  # avoid log(0)

    if bandwidth <= 0:
        raise ValueError("Bandwidth must be positive.")

    log_gabor = np.exp(-((np.log(D / center_freq)) ** 2) / (2 * (np.log(bandwidth)) ** 2))
    log_gabor /= np.max(log_gabor)  # normalize to max 1

    return log_gabor.astype(np.float32)


# example
filter_log_gabor_1 = log_gabor_filter((256, 256), center_freq=10, bandwidth=1.5)
filter_log_gabor_2 = log_gabor_filter((256, 256), center_freq=20, bandwidth=2.0)
filter_log_gabor_3 = log_gabor_filter((256, 256), center_freq=30, bandwidth=2.5)
filter_log_gabor_4 = log_gabor_filter((256, 256), center_freq=40, bandwidth=3.0)

# plot
kernels = [filter_log_gabor_1, filter_log_gabor_2, filter_log_gabor_3, filter_log_gabor_4]
titles = [
    "center_freq=10, bandwidth=0.5",
    "center_freq=20, bandwidth=0.75",
    "center_freq=30, bandwidth=1.0",
    "center_freq=40, bandwidth=1.5",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

##### <a id='toc3_1_1_5_4_'></a>[Band-Reject Filters](#toc0_)


In [ ]:
# ideal band-reject filter (IBRF)
def ideal_band_reject_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    IBRF(u,v) = 1.0 - IBPF(u,v; D1, D2)
    """
    IBPF = ideal_band_pass_filter(shape, D1, D2)
    IBRF = 1.0 - IBPF
    return IBRF


# example
filter_ibrf_1 = ideal_band_reject_filter(im_1.shape, D1=16, D2=64)
filter_ibrf_2 = ideal_band_reject_filter(im_1.shape, D1=32, D2=64)
filter_ibrf_3 = ideal_band_reject_filter(im_1.shape, D1=32, D2=128)
filter_ibrf_4 = ideal_band_reject_filter(im_1.shape, D1=64, D2=128)

# plot
kernels = [filter_ibrf_1, filter_ibrf_2, filter_ibrf_3, filter_ibrf_4]
titles = ["D1=16, D2=64", "D1=32, D2=64", "D1=32, D2=128", "D1=64, D2=128"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gaussian band-reject filter (GBRF)
def gaussian_band_reject_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    GBRF(u,v) = 1.0 - GBPF(u,v; D1, D2)
    """
    GBPF = gaussian_band_pass_filter(shape, D1, D2)
    GBRF = 1.0 - GBPF
    return GBRF


# example
filter_gbrf_1 = gaussian_band_reject_filter(im_1.shape, D1=16, D2=32)
filter_gbrf_2 = gaussian_band_reject_filter(im_1.shape, D1=32, D2=64)
filter_gbrf_3 = gaussian_band_reject_filter(im_1.shape, D1=32, D2=64)
filter_gbrf_4 = gaussian_band_reject_filter(im_1.shape, D1=64, D2=100)

# plot
kernels = [filter_gbrf_1, filter_gbrf_2, filter_gbrf_3, filter_gbrf_4]
titles = ["D1=16, D2=32", "D1=32, D2=64", "D1=32, D2=64", "D1=64, D2=100"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# butterworth band-reject filter (BBRF)
def butterworth_band_reject_filter(shape: tuple[int, int], D1: float, D2: float, order: int = 2) -> np.ndarray:
    """
    BBRF(u,v) = 1 - BBPF(u,v; D1, D2, order)
    """
    BBPF = butterworth_band_pass_filter(shape, D1, D2, order)
    BBRF = 1.0 - BBPF
    return BBRF


# example
filter_bbrf_1 = butterworth_band_reject_filter(im_1.shape, D1=16, D2=32, order=2)
filter_bbrf_2 = butterworth_band_reject_filter(im_1.shape, D1=32, D2=64, order=2)
filter_bbrf_3 = butterworth_band_reject_filter(im_1.shape, D1=32, D2=64, order=8)
filter_bbrf_4 = butterworth_band_reject_filter(im_1.shape, D1=64, D2=100, order=8)

# plot
kernels = [filter_bbrf_1, filter_bbrf_2, filter_bbrf_3, filter_bbrf_4]
titles = ["D1=16, D2=32, order=2", "D1=32, D2=64, order=2", "D1=32, D2=64, order=8", "D1=64, D2=100, order=8"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# exponential band-reject filter (EBRF)
def exponential_band_reject_filter(shape: tuple[int, int], D1: float, D2: float) -> np.ndarray:
    """
    EBRF(u,v) = 1 - EBPF(u,v; D1, D2)
    """
    EBPF = exponential_band_pass_filter(shape, D1, D2)
    EBRF = 1.0 - EBPF
    return EBRF


# example
filter_ebrf_1 = exponential_band_reject_filter(im_1.shape, D1=16, D2=32)
filter_ebrf_2 = exponential_band_reject_filter(im_1.shape, D1=32, D2=64)
filter_ebrf_3 = exponential_band_reject_filter(im_1.shape, D1=32, D2=64)
filter_ebrf_4 = exponential_band_reject_filter(im_1.shape, D1=64, D2=100)

# plot
kernels = [filter_ebrf_1, filter_ebrf_2, filter_ebrf_3, filter_ebrf_4]
titles = ["D1=16, D2=32", "D1=32, D2=64", "D1=32, D2=64", "D1=64, D2=100"]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# trapezoidal band-reject filter (TBRF)
def trapezoidal_band_reject_filter(shape: tuple[int, int], D1: float, D2: float, D3: float, D4: float) -> np.ndarray:
    """
    TBRF(u,v) = 1 - TBPF(u,v; D1, D2, D3, D4)
    """
    TBPF = trapezoidal_band_pass_filter(shape, D1, D2, D3, D4)
    TBRF = 1.0 - TBPF
    return TBRF


# example
filter_tbrf_1 = trapezoidal_band_reject_filter(im_1.shape, D1=8, D2=16, D3=32, D4=64)
filter_tbrf_2 = trapezoidal_band_reject_filter(im_1.shape, D1=16, D2=32, D3=40, D4=72)
filter_tbrf_3 = trapezoidal_band_reject_filter(im_1.shape, D1=16, D2=32, D3=64, D4=100)
filter_tbrf_4 = trapezoidal_band_reject_filter(im_1.shape, D1=32, D2=64, D3=76, D4=112)

# plot
kernels = [filter_tbrf_1, filter_tbrf_2, filter_tbrf_3, filter_tbrf_4]
titles = [
    "D1=8, D2=16, D3=32, D4=64",
    "D1=16, D2=32, D3=40, D4=72",
    "D1=16, D2=32, D3=64, D4=100",
    "D1=32, D2=64, D3=76, D4=112",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# notch filter
def notch_filter(shape: tuple[int, int], center: tuple[int, int], radius: float) -> np.ndarray:
    """
    Zeros out frequencies within radius around the center (u_center, v_center).
    Passes all other frequencies.
    """
    rows, cols = shape
    v_center, u_center = center
    u, v = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((u - u_center) ** 2 + (v - v_center) ** 2)
    notch = np.ones_like(D)
    notch[D <= radius] = 0
    return notch


# example
filter_notch_1 = notch_filter(im_1.shape, center=(32, 32), radius=8)
filter_notch_2 = notch_filter(im_1.shape, center=(32, 64), radius=16)
filter_notch_3 = notch_filter(im_1.shape, center=(128, 128), radius=32)
filter_notch_4 = notch_filter(im_1.shape, center=(140, 160), radius=64)

# plot
kernels = [filter_notch_1, filter_notch_2, filter_notch_3, filter_notch_4]
titles = [
    "center=(32,32), radius=8",
    "center=(32,64), radius=16",
    "center=(128,128), radius=32",
    "center=(140,160), radius=64",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

In [ ]:
# gaussian notch filter
def gaussian_notch_filter(shape: tuple[int, int], center: tuple[int, int], sigma: float) -> np.ndarray:
    """
    H(u,v) = 1 - exp(-((u - u_c)^2 + (v - v_c)^2) / (2 * sigma^2))
    """
    rows, cols = shape
    u_center, v_center = center
    u, v = np.meshgrid(np.arange(cols), np.arange(rows))
    D = np.sqrt((u - u_center) ** 2 + (v - v_center) ** 2)
    notch = 1.0 - np.exp(-(D**2) / (2 * sigma**2))
    return notch


# example
filter_gaussian_notch_1 = gaussian_notch_filter(im_1.shape, center=(32, 32), sigma=1)
filter_gaussian_notch_2 = gaussian_notch_filter(im_1.shape, center=(32, 64), sigma=2)
filter_gaussian_notch_3 = gaussian_notch_filter(im_1.shape, center=(128, 128), sigma=4)
filter_gaussian_notch_4 = gaussian_notch_filter(im_1.shape, center=(140, 160), sigma=8)

# plot
kernels = [filter_gaussian_notch_1, filter_gaussian_notch_2, filter_gaussian_notch_3, filter_gaussian_notch_4]
titles = [
    "center=(32,32), sigma=1",
    "center=(32,64), sigma=2",
    "center=(128,128), sigma=4",
    "center=(140,160), sigma=8",
]
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5), layout="compressed")
for i in range(len(kernels)):
    axs[i].imshow(kernels[i], cmap="gray")
    axs[i].set(title=titles[i])
plt.show()

#### <a id='toc3_1_1_6_'></a>[Effects on Fourier Spectrum](#toc0_)


##### <a id='toc3_1_1_6_1_'></a>[Flip](#toc0_)

**Horizontal Flip (Left-Right Flip):**

$$|F_{flipped}(u, v)| = |F(u, -v)| = |F(u, v)| \quad , \quad φ_{flipped}(u, v) = -φ(u, -v)$$

**Vertical Flip (Up-Down Flip):**

$$|F_{flipped}(u, v)| = |F(-u, v)| = |F(u, v)| \quad , \quad φ_{flipped}(u, v) = -φ(-u, v)$$


**Diagonal Flip (Rotate 180):**

$$|F_{flipped}(u, v)| = |F(-u, -v)| = |F(u, v)| \quad , \quad φ_{flipped}(u, v) = φ(-u, -v) = φ(u, v)$$


In [ ]:
im_1_flip_1 = cv2.flip(im_1, 1)
im_1_flip_2 = cv2.flip(im_1, 0)
im_1_flip_3 = cv2.flip(im_1, -1)

# dft
im_1_fft = np.fft.fftshift(np.fft.fft2(im_1))
im_1_flip_1_fft = np.fft.fftshift(np.fft.fft2(im_1_flip_1))
im_1_flip_2_fft = np.fft.fftshift(np.fft.fft2(im_1_flip_2))
im_1_flip_3_fft = np.fft.fftshift(np.fft.fft2(im_1_flip_3))

# magnitude (dB)
im_1_fft_mag = 20 * np.log10(np.abs(im_1_fft) + 1e-12)
im_1_flip_1_fft_mag = 20 * np.log10(np.abs(im_1_flip_1_fft) + 1e-12)
im_1_flip_2_fft_mag = 20 * np.log10(np.abs(im_1_flip_2_fft) + 1e-12)
im_1_flip_3_fft_mag = 20 * np.log10(np.abs(im_1_flip_3_fft) + 1e-12)

# phase
im_1_fft_pha = np.angle(im_1_fft)
im_1_flip_1_fft_pha = np.angle(im_1_flip_1_fft)
im_1_flip_2_fft_pha = np.angle(im_1_flip_2_fft)
im_1_flip_3_fft_pha = np.angle(im_1_flip_3_fft)


# plot
images = [
    [im_1, im_1_flip_1, im_1_flip_2, im_1_flip_3],
    [im_1_fft_mag, im_1_flip_1_fft_mag, im_1_flip_2_fft_mag, im_1_flip_3_fft_mag],
    [im_1_fft_pha, im_1_flip_1_fft_pha, im_1_flip_2_fft_pha, im_1_flip_3_fft_pha],
]
titles = [
    ["im_1", "im_1_flip_1", "im_1_flip_2", "im_1_flip_3"],
    ["Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)"],
    ["Phase", "Phase", "Phase", "Phase"],
]
fig, axs = plt.subplots(nrows=len(images), ncols=len(images[0]), figsize=(12, 12), layout="compressed")
for i in range(len(images)):
    for j in range(len(images[0])):
        if i == 2:  # phase row index
            axs[i, j].imshow(images[i][j], cmap="twilight", vmin=-np.pi, vmax=np.pi)
        else:
            axs[i, j].imshow(images[i][j], cmap="gray")
        axs[i, j].set(title=titles[i][j], xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_6_2_'></a>[Circular Shift](#toc0_)

- The magnitude of the Fourier Transform is unchanged by a circular shift.
- The phase of the Fourier Transform is modified by a circular shift.


In [ ]:
img_1_shift_1 = np.roll(im_1, shift=(128, 0), axis=(0, 1))
img_1_shift_2 = np.roll(im_1, shift=(0, 128), axis=(0, 1))
img_1_shift_3 = np.roll(im_1, shift=(128, 128), axis=(0, 1))


# dft
img_1_fft = np.fft.fftshift(np.fft.fft2(im_1))
img_1_shift_1_fft = np.fft.fftshift(np.fft.fft2(img_1_shift_1))
img_1_shift_2_fft = np.fft.fftshift(np.fft.fft2(img_1_shift_2))
img_1_shift_3_fft = np.fft.fftshift(np.fft.fft2(img_1_shift_3))

# magnitude (dB)
img_1_fft_mag = 20 * np.log10(np.abs(img_1_fft) + 1e-12)
img_1_shift_1_fft_mag = 20 * np.log10(np.abs(img_1_shift_1_fft) + 1e-12)
img_1_shift_2_fft_mag = 20 * np.log10(np.abs(img_1_shift_2_fft) + 1e-12)
img_1_shift_3_fft_mag = 20 * np.log10(np.abs(img_1_shift_3_fft) + 1e-12)

# phase
img_1_fft_pha = np.angle(img_1_fft)
img_1_shift_1_fft_pha = np.angle(img_1_shift_1_fft)
img_1_shift_2_fft_pha = np.angle(img_1_shift_2_fft)
img_1_shift_3_fft_pha = np.angle(img_1_shift_3_fft)


# plot
images = [
    [im_1, img_1_shift_1, img_1_shift_2, img_1_shift_3],
    [img_1_fft_mag, img_1_shift_1_fft_mag, img_1_shift_2_fft_mag, img_1_shift_3_fft_mag],
    [img_1_fft_pha, img_1_shift_1_fft_pha, img_1_shift_2_fft_pha, img_1_shift_3_fft_pha],
]
titles = [
    ["im_1", "img_1_shift_1", "img_1_shift_2", "img_1_shift_3"],
    ["Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)"],
    ["Phase", "Phase", "Phase", "Phase"],
]
fig, axs = plt.subplots(nrows=len(images), ncols=len(images[0]), figsize=(12, 12), layout="compressed")
for i in range(len(images)):
    for j in range(len(images[0])):
        if i == 2:  # phase row index
            axs[i, j].imshow(images[i][j], cmap="twilight", vmin=-np.pi, vmax=np.pi)
        else:
            axs[i, j].imshow(images[i][j], cmap="gray")
        axs[i, j].set(title=titles[i][j], xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_6_3_'></a>[Rotate](#toc0_)

- Rotating an image by $n$ degrees in the spatial domain rotates both the **magnitude** and **phase** of its Fourier Transform by $n$ degrees in the **frequency domain**.

$$|F_{\text{rotated}}(u, v)| = |F(u \cos \theta - v \sin \theta, u \sin \theta + v \cos \theta)| \quad , \quad \phi_{\text{rotated}}(u, v) = \phi(u \cos \theta - v \sin \theta, u \sin \theta + v \cos \theta)$$


In [ ]:
img_1_rotate_1 = np.rot90(im_1, 1)
img_1_rotate_2 = np.rot90(im_1, 2)
img_1_rotate_3 = np.rot90(im_1, 3)


# dft
img_1_fft = np.fft.fftshift(np.fft.fft2(im_1))
img_1_rotate_1_fft = np.fft.fftshift(np.fft.fft2(img_1_rotate_1))
img_1_rotate_2_fft = np.fft.fftshift(np.fft.fft2(img_1_rotate_2))
img_1_rotate_3_fft = np.fft.fftshift(np.fft.fft2(img_1_rotate_3))

# magnitude (dB)
img_1_fft_mag = 20 * np.log10(np.abs(img_1_fft) + 1e-12)
img_1_rotate_1_fft_mag = 20 * np.log10(np.abs(img_1_rotate_1_fft) + 1e-12)
img_1_rotate_2_fft_mag = 20 * np.log10(np.abs(img_1_rotate_2_fft) + 1e-12)
img_1_rotate_3_fft_mag = 20 * np.log10(np.abs(img_1_rotate_3_fft) + 1e-12)

# phase
img_1_fft_pha = np.angle(img_1_fft)
img_1_rotate_1_fft_pha = np.angle(img_1_rotate_1_fft)
img_1_rotate_2_fft_pha = np.angle(img_1_rotate_2_fft)
img_1_rotate_3_fft_pha = np.angle(img_1_rotate_3_fft)


# plot
images = [
    [im_1, img_1_rotate_1, img_1_rotate_2, img_1_rotate_3],
    [img_1_fft_mag, img_1_rotate_1_fft_mag, img_1_rotate_2_fft_mag, img_1_rotate_3_fft_mag],
    [img_1_fft_pha, img_1_rotate_1_fft_pha, img_1_rotate_2_fft_pha, img_1_rotate_3_fft_pha],
]
titles = [
    ["im_1", "img_1_rotate_1", "img_1_rotate_2", "img_1_rotate_3"],
    ["Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)", "Magnitude (dB)"],
    ["Phase", "Phase", "Phase", "Phase"],
]
fig, axs = plt.subplots(nrows=len(images), ncols=len(images[0]), figsize=(12, 12), layout="compressed")
for i in range(len(images)):
    for j in range(len(images[0])):
        if i == 2:  # phase row index
            axs[i, j].imshow(images[i][j], cmap="twilight", vmin=-np.pi, vmax=np.pi)
        else:
            axs[i, j].imshow(images[i][j], cmap="gray")
        axs[i, j].set(title=titles[i][j], xticks=[], yticks=[])
plt.show()

#### <a id='toc3_1_1_7_'></a>[Examples](#toc0_)


##### <a id='toc3_1_1_7_1_'></a>[Example 1: Exploring Ringing Effect](#toc0_)


In [ ]:
filter_ilpf_1 = ideal_low_pass_filter(shape=im_1.shape, cutoff=16)
filter_ilpf_2 = ideal_low_pass_filter(shape=im_1.shape, cutoff=32)
filter_ilpf_3 = ideal_low_pass_filter(shape=im_1.shape, cutoff=64)

In [ ]:
# fft
im_1_gaussian_noise_fft = np.fft.fftshift(np.fft.fft2(im_1_gaussian_noise))

# apply filters
im_1_apply_ilpf_1 = im_1_gaussian_noise_fft * filter_ilpf_1
im_1_apply_ilpf_2 = im_1_gaussian_noise_fft * filter_ilpf_2
im_1_apply_ilpf_3 = im_1_gaussian_noise_fft * filter_ilpf_3

# reconstruction
im_1_apply_ilpf_ifft_1 = np.fft.ifft2(np.fft.ifftshift(im_1_apply_ilpf_1))
im_1_apply_ilpf_ifft_2 = np.fft.ifft2(np.fft.ifftshift(im_1_apply_ilpf_2))
im_1_apply_ilpf_ifft_3 = np.fft.ifft2(np.fft.ifftshift(im_1_apply_ilpf_3))

# clip the range & convert dtype
im_1_apply_ilpf_ifft_1 = np.clip(im_1_apply_ilpf_ifft_1.real, 0, 255).astype(np.uint8)
im_1_apply_ilpf_ifft_2 = np.clip(im_1_apply_ilpf_ifft_2.real, 0, 255).astype(np.uint8)
im_1_apply_ilpf_ifft_3 = np.clip(im_1_apply_ilpf_ifft_3.real, 0, 255).astype(np.uint8)

In [ ]:
# plot
images = [im_1_gaussian_noise, im_1_apply_ilpf_ifft_1, im_1_apply_ilpf_ifft_2, im_1_apply_ilpf_ifft_3]
titles = ["Original", "cutoff=16", "cutoff=32", "cutoff=64"]
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(16, 8), layout="compressed")
for i in range(len(images)):
    axs[0, i].imshow(images[i], cmap="gray", vmin=0, vmax=255)
    axs[0, i].set(title=titles[i], xticks=[], yticks=[])
    axs[1, i].imshow(images[i][:80, 70:150], cmap="gray", vmin=0, vmax=255)
    axs[1, i].set(title=titles[i], xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_7_2_'></a>[Example 2: Image Smoothing](#toc0_)


In [ ]:
filter_glpf = gaussian_low_pass_filter(shape=im_1.shape, sigma=32)

In [ ]:
# fft
im_1_gaussian_noise_fft = np.fft.fftshift(np.fft.fft2(im_1_gaussian_noise))

# before applying the filter
im_1_fft_mag = 20 * np.log10(np.abs(im_1_gaussian_noise_fft) + 1e-12)
im_1_fft_pha = np.angle(im_1_gaussian_noise_fft)

# after applying the filter
im_1_apply_glpf = im_1_gaussian_noise_fft * filter_glpf
im_1_apply_glpf_mag = 20 * np.log10(np.abs(im_1_apply_glpf) + 1e-12)
im_1_apply_glpf_pha = np.angle(im_1_apply_glpf)

# reconstruction
im_1_apply_glpf_ifft = np.fft.ifft2(np.fft.ifftshift(im_1_apply_glpf))
im_1_apply_glpf_ifft = np.clip(im_1_apply_glpf_ifft.real, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1_gaussian_noise, cmap="gray")
axs[0, 0].set(title="Original", xticks=[], yticks=[])
axs[0, 1].imshow(im_1_fft_mag, cmap="gray")
axs[0, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[0, 2].imshow(im_1_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[0, 2].set(title="Phase", xticks=[], yticks=[])
axs[1, 0].imshow(im_1_apply_glpf_ifft, cmap="gray")
axs[1, 0].set(title="Reconstructed", xticks=[], yticks=[])
axs[1, 1].imshow(im_1_apply_glpf_mag, cmap="gray", vmin=im_1_fft_mag.min(), vmax=im_1_fft_mag.max())
axs[1, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[1, 2].imshow(im_1_apply_glpf_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[1, 2].set(title="Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_7_3_'></a>[Example 3: Image Sharpenning](#toc0_)


In [ ]:
filter_lhpf = laplacian_high_pass_filter(im_1.shape)

In [ ]:
# fft
im_1_gaussian_noise_fft = np.fft.fftshift(np.fft.fft2(im_1_gaussian_noise))

# before applying the filter
im_1_fft_mag = 20 * np.log10(np.abs(im_1_gaussian_noise_fft) + 1e-12)
im_1_fft_pha = np.angle(im_1_gaussian_noise_fft)

# after applying the filter
im_1_apply_lhpf = im_1_gaussian_noise_fft * filter_lhpf
im_1_apply_lhpf_mag = 20 * np.log10(np.abs(im_1_apply_lhpf) + 1e-12)
im_1_apply_lhpf_pha = np.angle(im_1_apply_lhpf)

# reconstruction
im_1_apply_lhpf_ifft = np.fft.ifft2(np.fft.ifftshift(im_1_apply_lhpf))
im_1_apply_lhpf_ifft = np.clip(im_1_apply_lhpf_ifft.real, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1_gaussian_noise, cmap="gray")
axs[0, 0].set(title="Original", xticks=[], yticks=[])
axs[0, 1].imshow(im_1_fft_mag, cmap="gray")
axs[0, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[0, 2].imshow(im_1_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[0, 2].set(title="Phase", xticks=[], yticks=[])
axs[1, 0].imshow(im_1_apply_lhpf_ifft * 5, cmap="gray")
axs[1, 0].set(title="Reconstructed * 5", xticks=[], yticks=[])
axs[1, 1].imshow(im_1_apply_lhpf_mag, cmap="gray", vmin=im_1_fft_mag.min(), vmax=im_1_fft_mag.max())
axs[1, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[1, 2].imshow(im_1_apply_lhpf_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[1, 2].set(title="Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_7_4_'></a>[Example 4: Edge Detection](#toc0_)


In [ ]:
filter_sobel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

In [ ]:
# pad filter and image
filter_sobel_padded = np.pad(filter_sobel, pad_width=((0, im_1.shape[0] - 1), (0, im_1.shape[1] - 1)))
im_1_gaussian_noise_padded = np.pad(
    im_1_gaussian_noise, pad_width=((0, filter_sobel.shape[0] - 1), (0, filter_sobel.shape[1] - 1))
)

# fft
filter_sobel_fft = np.fft.fftshift(np.fft.fft2(filter_sobel_padded))
im_1_gaussian_noise_fft = np.fft.fftshift(np.fft.fft2(im_1_gaussian_noise_padded))

# before applying the filter
im_1_fft_mag = 20 * np.log10(np.abs(im_1_gaussian_noise_fft) + 1e-12)
im_1_fft_pha = np.angle(im_1_gaussian_noise_fft)

# after applying the filter
im_1_apply_sobel = im_1_gaussian_noise_fft * filter_sobel_fft
im_1_apply_sobel_mag = 20 * np.log10(np.abs(im_1_apply_sobel) + 1e-12)
im_1_apply_sobel_pha = np.angle(im_1_apply_sobel)

# reconstruction
im_1_apply_sobel_ifft = np.fft.ifft2(np.fft.ifftshift(im_1_apply_sobel))
im_1_apply_sobel_ifft = im_1_apply_sobel_ifft[: im_1.shape[0], : im_1.shape[1]]
im_1_apply_sobel_ifft = np.clip(im_1_apply_sobel_ifft.real, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1_gaussian_noise, cmap="gray")
axs[0, 0].set(title="Original", xticks=[], yticks=[])
axs[0, 1].imshow(im_1_fft_mag, cmap="gray")
axs[0, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[0, 2].imshow(im_1_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[0, 2].set(title="Phase", xticks=[], yticks=[])
axs[1, 0].imshow(im_1_apply_sobel_ifft * 5, cmap="gray")
axs[1, 0].set(title="Reconstructed * 5", xticks=[], yticks=[])
axs[1, 1].imshow(im_1_apply_sobel_mag, cmap="gray", vmin=im_1_fft_mag.min(), vmax=im_1_fft_mag.max())
axs[1, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[1, 2].imshow(im_1_apply_sobel_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[1, 2].set(title="Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_7_5_'></a>[Example 5: Periodic Noise Removal](#toc0_)


In [ ]:
# fft
im_1_periodic_noise_fft = np.fft.fftshift(np.fft.fft2(im_1_periodic_noise))

# magnitude (dB) & phase
im_1_fft_mag = 20 * np.log10(np.abs(im_1_periodic_noise_fft) + 1e-12)
im_1_fft_pha = np.angle(im_1_periodic_noise_fft)

# plot
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 8), layout="compressed")
axs[0].imshow(im_1_periodic_noise, cmap="gray")
axs[0].set(title="Original", xticks=[], yticks=[])
axs[1].imshow(im_1_fft_mag, cmap="gray")
axs[1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[2].imshow(im_1_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[2].set(title="Phase", xticks=[], yticks=[])
plt.show()

In [ ]:
# find noise peaks
peaks, _ = sp.signal.find_peaks(im_1_fft_mag.ravel())
top_n_indices = np.argsort(im_1_fft_mag.ravel()[peaks])[-19:][::-1]
peak_coords = [np.unravel_index(peaks[i], im_1_fft_mag.shape) for i in top_n_indices]

# log
for i in range(len(peak_coords)):
    print(peak_coords[i])

In [ ]:
# define notch filters
manual_coords = [[119, 109], [137, 147], [108, 89], [148, 167], [158, 188], [98, 68]]
notch_filters = [notch_filter(shape=im_1.shape, center=coord, radius=2) for coord in manual_coords]

# apply notch filters
im_1_apply_notch = im_1_periodic_noise_fft.copy()
for notch in notch_filters:
    im_1_apply_notch = im_1_apply_notch * notch

# magnitude (dB) & phase after applying notch filters
im_1_apply_notch_mag = 20 * np.log10(np.abs(im_1_apply_notch) + 1e-12)
im_1_apply_notch_pha = np.angle(im_1_apply_notch)

# reconstruction
im_1_apply_notch_ifft = np.fft.ifft2(np.fft.ifftshift(im_1_apply_notch))
im_1_apply_notch_ifft = np.clip(im_1_apply_notch_ifft.real, 0, 255).astype(np.uint8)

# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1_periodic_noise, cmap="gray")
axs[0, 0].set(title="Original", xticks=[], yticks=[])
axs[0, 1].imshow(im_1_fft_mag, cmap="gray")
axs[0, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[0, 2].imshow(im_1_fft_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[0, 2].set(title="Phase", xticks=[], yticks=[])
axs[1, 0].imshow(im_1_apply_notch_ifft, cmap="gray")
axs[1, 0].set(title="Reconstructed", xticks=[], yticks=[])
axs[1, 1].imshow(im_1_apply_notch_mag, cmap="gray", vmin=im_1_fft_mag.min(), vmax=im_1_fft_mag.max())
axs[1, 1].set(title="Magnitude (dB)", xticks=[], yticks=[])
axs[1, 2].imshow(im_1_apply_notch_pha, cmap="twilight", vmin=-np.pi, vmax=np.pi)
axs[1, 2].set(title="Phase", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_1_7_6_'></a>[Example 6: Incomplete Reconstruction](#toc0_)


In [ ]:
# fft
im_1_fft = np.fft.fftshift(np.fft.fft2(im_1))
im_1_fft_mag = np.abs(im_1_fft)
im_1_fft_pha = np.angle(im_1_fft)

# reconstructed using magnitude
im_1_ifft_only_mag = np.clip(np.fft.ifft2(np.fft.ifftshift(im_1_fft_mag * np.exp(0j * im_1_fft_pha))).real, 0, 255)

# reconstructed using phase
im_1_ifft_only_pha = np.fft.ifft2(np.fft.ifftshift(np.exp(1j * im_1_fft_pha))).real

# plot
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 6), layout="compressed")
axs[0].imshow(im_1, cmap="gray")
axs[0].set(title=f"Original", xticks=[], yticks=[])
axs[1].imshow(im_1_ifft_only_mag, cmap="gray")
axs[1].set(title=f"Reconstructed [Only Magnitude]", xticks=[], yticks=[])
axs[2].imshow(im_1_ifft_only_pha, cmap="gray")
axs[2].set(title=f"Reconstructed [Only Phase]", xticks=[], yticks=[])
plt.show()

### <a id='toc3_1_2_'></a>[Discrete Cosine Transform (DCT)](#toc0_)

#### <a id='toc3_1_2_1_'></a>[2-Dimensional Signal](#toc0_)

##### <a id='toc3_1_2_1_1_'></a>[Even-Symmetric Signal](#toc0_)

- For a 2D signal $f(x,y)$, even-symmetry means the signal is symmetric across both $x\text{-axis}$ and $y\text{-axis}$:

$$f(x,y)=f(-x,-y)$$


In [ ]:
signal_periodic = np.tile(im_1, (4, 4))
signal_even_symmetric_periodic = np.tile(np.pad(im_1, (0, im_1.shape[0]), mode="symmetric"), (2, 2))

# plot
fig, axes = plt.subplots(1, 3, figsize=(20, 6), layout="compressed")
axes[0].imshow(im_1, cmap="gray")
axes[0].set(title="Original Signal", xticks=[], yticks=[])
axes[1].imshow(signal_periodic, cmap="gray")
axes[1].set(title="Fourier: Periodic Signal", xticks=[], yticks=[])
axes[2].imshow(signal_even_symmetric_periodic, cmap="gray")
axes[2].set(title="Cosine: Even-Symmetric & Periodic Signal", xticks=[], yticks=[])
plt.show()

##### <a id='toc3_1_2_1_2_'></a>[Basis Images](#toc0_)

we now have two sets of basis vectors—one corresponding to the rows and the other to the columns.

**Mathematical Formula:**

- Let $f(x,y)$ be a 2D signal of size $N \times M$. The 2D DCT matrix $C$ can be computed as:
$$C_{k_1,k_2,x,y} = \cos\left[\frac{\pi}{N} \left(k_1 + \frac{1}{2}\right)x\right] \cos\left[\frac{\pi}{M} \left(k_2 + \frac{1}{2}\right)y\right]$$

- For $N=4, M=5$, the DCT matrix is:
$$C_{k_1, k_2} =
\begin{bmatrix}
C_{0, 0} & C_{0, 1} & \cdots & C_{0, M-1} \\
C_{1, 0} & C_{1, 1} & \cdots & C_{1, M-1} \\
\vdots & \vdots & \ddots & \vdots \\
C_{N-1, 0} & C_{N-1, 1} & \cdots & C_{N-1, M-1}
\end{bmatrix}, \quad
C_{0,0} =
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

- We can build the **2D basis images** by taking the **outer product** of the **1D basis vectors** for **rows** and **columns**.
  - 1D DCT Basis Vector (Row Direction):
    $$v_1(k_1, x) = \cos\left[\frac{\pi}{N} \left(k_1 + \frac{1}{2}\right)x\right],  \quad x = 0, 1, \dots, N-1$$

  - 1D DCT Basis Vector (Column Direction):
    $$v_2(k_2, y) = \cos\left[\frac{\pi}{M} \left(k_2 + \frac{1}{2}\right)y\right],  \quad y = 0, 1, \dots, M-1$$

  - Constructing the 2D Basis Images:
    $$C_{k_1, k_2}(x, y) = v_1(k_1, x) \cdot v_2(k_2, y) = \cos\left[\frac{\pi}{N} \left(k_1 + \frac{1}{2}\right)x\right] \cos\left[\frac{\pi}{M} \left(k_2 + \frac{1}{2}\right)y\right]$$

**Example**

Consider the following $3 \times 4$ signal:

$$
S = 
\begin{bmatrix}
s_{0, 0} & s_{0, 1} & s_{0, 2} & s_{0, 3} \\
s_{1, 0} & s_{1, 1} & s_{1, 2} & s_{1, 3} \\
s_{2, 0} & s_{2, 1} & s_{2, 2} & s_{2, 3}
\end{bmatrix}
$$

The goal is to find the basis image $C_{1, 2}$, corresponding to the frequency pair $k_1 = 1$ and $k_2 = 2$.

- **Step 1**: 1D Basis Vectors
  - Row Basis Vector $v_1(k_1, x)$ for $k_1 = 1$:
    - The row basis vector for $k_1 = 1$ is given by:
    $$
    v_1(1, x) = \cos\left[\frac{\pi}{3} \left(1 + \frac{1}{2}\right)x\right] \quad \text{for } x = 0, 1, 2
    $$
    This results in:
    $$
    v_1(1, x) = \begin{bmatrix}
    \cos(0) \\
    \cos\left(\frac{3\pi}{6}\right) \\
    \cos\left(\frac{6\pi}{6}\right)
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    \cos\left(\frac{\pi}{2}\right) \\
    \cos(\pi)
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    0 \\
    -1
    \end{bmatrix}
    $$

  - Column Basis Vector $v_2(k_2, y)$ for $k_2 = 2$:
    - The column basis vector for $k_2 = 2$ is given by:
    $$
    v_2(2, y) = \cos\left[\frac{\pi}{4} \left(2 + \frac{1}{2}\right)y\right] \quad \text{for } y = 0, 1, 2, 3
    $$
    This results in:
    $$
    v_2(2, y) = \begin{bmatrix}
    \cos(0) \\
    \cos\left(\frac{10\pi}{16}\right) \\
    \cos\left(\frac{20\pi}{16}\right) \\
    \cos\left(\frac{30\pi}{16}\right)
    \end{bmatrix}
    = \begin{bmatrix}
    1 \\
    \cos\left(\frac{5\pi}{8}\right) \\
    \cos\left(\frac{10\pi}{8}\right) \\
    \cos\left(\frac{15\pi}{8}\right)
    \end{bmatrix}
    $$

- **Step 2**: Outer Product of 1D Basis Vectors
  - To compute the 2D basis image $C_{1, 2}(x, y)$, we take the outer product of $v_1(1, x)$ and $v_2(2, y)$:
    $$
    C_{1, 2}(x, y) = v_1(1, x) \cdot v_2(2, y)
    $$

    The result of the outer product is:

    $$
    C_{1, 2} = 
    \begin{bmatrix}
    1 \cdot 1 & 1 \cdot \cos\left(\frac{5\pi}{8}\right) & 1 \cdot \cos\left(\frac{10\pi}{8}\right) & 1 \cdot \cos\left(\frac{15\pi}{8}\right) \\
    0 \cdot 1 & 0 \cdot \cos\left(\frac{5\pi}{8}\right) & 0 \cdot \cos\left(\frac{10\pi}{8}\right) & 0 \cdot \cos\left(\frac{15\pi}{8}\right) \\
    -1 \cdot 1 & -1 \cdot \cos\left(\frac{5\pi}{8}\right) & -1 \cdot \cos\left(\frac{10\pi}{8}\right) & -1 \cdot \cos\left(\frac{15\pi}{8}\right)
    \end{bmatrix}
    $$

    Simplifying each term:

    $$
    C_{1, 2} = 
    \begin{bmatrix}
    1 & \cos\left(\frac{5\pi}{8}\right) & \cos\left(\frac{10\pi}{8}\right) & \cos\left(\frac{15\pi}{8}\right) \\
    0 & 0 & 0 & 0 \\
    -1 & -\cos\left(\frac{5\pi}{8}\right) & -\cos\left(\frac{10\pi}{8}\right) & -\cos\left(\frac{15\pi}{8}\right)
    \end{bmatrix}
    $$


In [ ]:
def dct_basis_vectors(length: int, mode: str = "efficient", normalization: bool = True) -> np.ndarray:
    if mode == "inefficient":
        C = np.zeros((length, length))
        for k in range(length):
            for x in range(length):
                C[k, x] = np.cos((np.pi / length) * (x + 0.5) * k)
    elif mode == "efficient":
        x = np.arange(length)
        k = x[:, None]
        C = np.cos((np.pi / length) * (x + 0.5) * k)

    if normalization:
        C[0, :] *= np.sqrt(1 / length)  # DC normalization
        C[1:, :] *= np.sqrt(2 / length)  # AC normalization

    return C

In [ ]:
def dct_basis_images(rows: int, cols: int, mode: str = "efficient", normalization: bool = True) -> np.ndarray:
    row_basis = dct_basis_vectors(rows, normalization=normalization)  # 1D basis vectors for rows
    col_basis = dct_basis_vectors(cols, normalization=normalization)  # 1D basis vectors for columns

    if mode == "inefficient":
        basis_images = np.zeros((rows, cols, rows, cols))  # 4D array to store the 2D basis images
        for k1 in range(rows):
            for k2 in range(cols):
                row_vector = row_basis[k1, :]
                col_vector = col_basis[k2, :]
                basis_images[k1, k2] = np.outer(row_vector, col_vector)

    elif mode == "efficient":
        basis_images = row_basis[:, None, :, None] * col_basis[None, :, None, :]

    return basis_images

In [ ]:
signal_row = 3
signal_col = 4
basis_images = dct_basis_images(signal_row, signal_col, normalization=False)

# log
print(f"basis_images.shape: {basis_images.shape}")
print(f"W_0,0:\n{basis_images[0, 0]}")

In [ ]:
signal_row = 8
signal_col = 8
basis_images = dct_basis_images(signal_row, signal_col, normalization=False)

# plot
fig, axes = plt.subplots(signal_row, signal_col, figsize=(10, 10), layout="compressed")
for i in range(signal_row):
    for j in range(signal_col):
        basis_image = basis_images[i, j]
        axes[i, j].imshow(basis_image, cmap="gray", vmin=-1, vmax=1)
        axes[i, j].axis("off")
for j in range(signal_col):
    axes[0, j].set_title(f"(0,{j+1})", fontsize=8)
plt.show()

In [ ]:
N = basis_images.shape[0]
M = basis_images.shape[1]
cross_correlation = np.einsum("ijkl,mnkl->ijmn", basis_images, basis_images, optimize=True)

# plot
fig, axes = plt.subplots(N, M, figsize=(M * 2, N * 2 + 1), layout="compressed")
for k1 in range(N):
    for l1 in range(M):
        axes[k1, l1].imshow(np.abs(cross_correlation[k1, l1]), cmap="gray")
        axes[k1, l1].set(title=f"({k1},{l1})", xticks=[], yticks=[])
        axes[k1, l1].set_xticks(np.arange(-0.5, cross_correlation.shape[3], 1), minor=True)
        axes[k1, l1].set_yticks(np.arange(-0.5, cross_correlation.shape[2], 1), minor=True)
        axes[k1, l1].grid(True, which="minor", color="white", linestyle="--", linewidth=0.5, alpha=0.5)
fig.suptitle("Cross-Correlation Between 8x8 DCT Basis Images")
plt.show()

##### <a id='toc3_1_2_1_3_'></a>[Forward 2D Discrete Cosine Transform (DCT2)](#toc0_)

- The 2D DCT can be computed by applying a 1D DCT along the rows followed by a 1D DCT along the columns.  
$$F = C_N \cdot f \cdot C_M^T$$  


In [ ]:
def dct2(image: np.ndarray) -> np.ndarray:
    rows, cols = image.shape
    C_row = dct_basis_vectors(rows)
    C_col = dct_basis_vectors(cols)
    F = C_row @ image @ C_col.T
    return F

##### <a id='toc3_1_2_1_4_'></a>[Inverse 2D Discrete Cosine Transform (IDCT2)](#toc0_)

- The 2D IDCT can be computed by applying a 1D IDCT along the columns followed by a 1D IDCT along the rows.  
$$f = C_N^T \cdot F \cdot C_M$$  


In [ ]:
def idct2(image: np.ndarray) -> np.ndarray:
    rows, cols = image.shape
    C_row = dct_basis_vectors(rows)
    C_col = dct_basis_vectors(cols)
    F = C_row.T @ image @ C_col
    return F

#### <a id='toc3_1_2_2_'></a>[Blocking Effect](#toc0_)


In [ ]:
patch_size = 32
patches = []

# extracting 32x32 patches
for i in range(0, im_1.shape[0], patch_size):
    for j in range(0, im_1.shape[1], patch_size):
        patch = im_1[i : i + patch_size, j : j + patch_size]
        patches.append(patch)

# plot
nrows = im_1.shape[0] // patch_size
ncols = im_1.shape[1] // patch_size
fig, axs = plt.subplots(nrows, ncols, figsize=(12, 12), layout="compressed")
for i in range(nrows):
    for j in range(ncols):
        axs[i, j].imshow(patches[i * ncols + j], cmap="gray", vmin=0, vmax=255)
        axs[i, j].axis("off")
plt.show()

In [ ]:
# appply DFT and DCT to each patch
dct_patches = []
dft_patches = []
for patch in patches:
    # apply DFT
    dft_patch = np.fft.fft2(patch)
    dft_patches.append(dft_patch)

    # apply DCT
    dct_patch = sp.fft.dctn(patch)
    dct_patches.append(dct_patch)

In [ ]:
# keep only a proportion of low frequencies (simulate compression)
prop = 0.2
dft_patches_reconstructed = []
dct_patches_reconstructed = []

for dct_patch, dft_patch in zip(dct_patches, dft_patches):
    threshold = int(prop * dct_patch.shape[0])

    # zero out high frequencies in DFT patch (both high rows and columns)
    dft_patch[threshold:, threshold:] = 0
    dft_patch = np.clip(np.fft.ifft2(dft_patch).real, 0, 255).astype(np.uint8)
    dft_patches_reconstructed.append(dft_patch)

    # zero out high frequencies in DCT patch
    dct_patch[threshold:, threshold:] = 0
    dct_patch = np.clip(sp.fft.idctn(dct_patch), 0, 255).astype(np.uint8)
    dct_patches_reconstructed.append(dct_patch)

In [ ]:
# reconstruct the full image by placing the reconstructed patches back into their positions
reconstructed_image_dft = np.zeros_like(im_1)
reconstructed_image_dct = np.zeros_like(im_1)

index = 0
for i in range(0, im_1.shape[0], patch_size):
    for j in range(0, im_1.shape[1], patch_size):
        reconstructed_image_dft[i : i + patch_size, j : j + patch_size] = dft_patches_reconstructed[index]
        reconstructed_image_dct[i : i + patch_size, j : j + patch_size] = dct_patches_reconstructed[index]
        index += 1

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 6), layout="compressed")
axs[0].imshow(im_1, cmap="gray")
axs[0].set_title("Original Image")
axs[0].axis("off")
axs[1].imshow(reconstructed_image_dft, cmap="gray")
axs[1].set_title(f"Reconstructed (DFT) - Prop: {prop*100}%")
axs[1].axis("off")
axs[2].imshow(reconstructed_image_dct, cmap="gray")
axs[2].set_title(f"Reconstructed (DCT) - Prop: {prop*100}%")
axs[2].axis("off")
plt.show()

### <a id='toc3_1_3_'></a>[Energy of the Signals](#toc0_)


In [ ]:
im_1 = im_1.astype(np.float64)
energy_im_1 = (im_1**2).sum().round()

# same energy
im_1_fft_1 = np.fft.fftshift(np.fft.fft2(im_1))
im_1_fft_1_mag = np.abs(im_1_fft_1)
energy_im_1_fft_1 = ((im_1_fft_1_mag**2).sum() / np.prod(im_1.shape)).round()

# increase energy
im_1_fft_2 = im_1_fft_1 * 10
im_1_fft_2_mag = np.abs(im_1_fft_2)
energy_im_1_fft_2 = ((im_1_fft_2_mag**2).sum() / np.prod(im_1.shape)).round()

# decrease energy
im_1_fft_3 = im_1_fft_1 / 100
im_1_fft_3_mag = np.abs(im_1_fft_3)
energy_im_1_fft_3 = ((im_1_fft_3_mag**2).sum() / np.prod(im_1.shape)).round()

# idft
im_1_ifft_1 = np.fft.ifft2(np.fft.ifftshift(im_1_fft_1))
im_1_ifft_2 = np.fft.ifft2(np.fft.ifftshift(im_1_fft_2))
im_1_ifft_3 = np.fft.ifft2(np.fft.ifftshift(im_1_fft_3))

energy_im_1_ifft_1 = (np.abs(im_1_ifft_1) ** 2).sum().round()
energy_im_1_ifft_2 = (np.abs(im_1_ifft_2) ** 2).sum().round()
energy_im_1_ifft_3 = (np.abs(im_1_ifft_3) ** 2).sum().round()

im_1_ifft_rec_1 = np.clip(im_1_ifft_1.real, 0, 255).astype(np.uint8)
im_1_ifft_rec_2 = np.clip(im_1_ifft_2.real, 0, 255).astype(np.uint8)
im_1_ifft_rec_3 = np.clip(im_1_ifft_3.real, 0, 255).astype(np.uint8)


# plot
fig = plt.figure(figsize=(18, 20), layout="constrained")
gs = GridSpec(nrows=3, ncols=3, figure=fig)
ax0 = fig.add_subplot(gs[0, :])
ax0.imshow(im_1, cmap="gray")
ax0.set(title=f"Original - Energy: {energy_im_1:,}", xticks=[], yticks=[])
ax1 = fig.add_subplot(gs[1, 0])
ax1.imshow(20 * np.log10(im_1_fft_1_mag + 1e-12), cmap="gray")
ax1.set(title=f"Energy: {energy_im_1_fft_1:,}", xticks=[], yticks=[])
ax2 = fig.add_subplot(gs[2, 0])
ax2.imshow(im_1_ifft_rec_1, cmap="gray")
ax2.set(title=f"Energy: {energy_im_1_ifft_1:,}", xticks=[], yticks=[])
ax3 = fig.add_subplot(gs[1, 1])
ax3.imshow(20 * np.log10(im_1_fft_2_mag + 1e-12), cmap="gray")
ax3.set(title=f"Energy: {energy_im_1_fft_2:,}", xticks=[], yticks=[])
ax4 = fig.add_subplot(gs[2, 1])
ax4.imshow(im_1_ifft_rec_2, cmap="gray")
ax4.set(title=f"Energy: {energy_im_1_ifft_2:,}", xticks=[], yticks=[])
ax5 = fig.add_subplot(gs[1, 2])
ax5.imshow(20 * np.log10(im_1_fft_3_mag + 1e-12), cmap="gray")
ax5.set(title=f"Energy: {energy_im_1_fft_3:,}", xticks=[], yticks=[])
ax6 = fig.add_subplot(gs[2, 2])
ax6.imshow(im_1_ifft_rec_3, cmap="gray")
ax6.set(title=f"Energy: {energy_im_1_ifft_3:,}", xticks=[], yticks=[])
plt.show()